In [1]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score

import pandas as pd

model = SetFitModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_trainset_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_testset_mod.xlsx")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)


trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=12,
    column_mapping={'Class':'label', 'Text':'text'},
    num_iterations=30, # Number of text pairs to generate for contrastive learning
    num_epochs=3 # Number of epochs to use for contrastive learning
)
start = time.process_time()
trainer.train()
runtime = time.process_time() - start

preds = trainer.model.predict([
    'Agent: youre more than welcome whats your name again alright  richard [[sys.name]]  thank you so much alright so i',
    'Agent: unlimited okay okay so in this case did you want to change the plan now or you want us to process the payment for the regular  40 dollars [[sys.money]] ',
    'Agent: hello welcome to cricket nation this is jake i how can i help you Customer: hi i was calling to do the payment in 2 im upgrading my my services well actually just to a higher my services to little bit more expensive for now much 30 dollars so i wanna  40 dollars [[sys.money]] ',
    'Agent: okay perfect im ready to process the payment may i please have the card number Customer: okay it is  4491710136 [[sys.number]] ',
    'Agent: because you have 3 lines on the 55 dollar plan your due date was actually yesterday but  11/50/9 PM [[sys.time]]  no this is',
    'Agent: okay so just inform them to re submit the order okay because that is the information that i see right now for the cellphone phone number aah ending again  1471 [[sys.number]]  there is the port protection so provide them again the aah aah aah port out number to port ahead to call this one the transfer pin and they will have to aah started over and reset make the order and i usually that should really work',
    'Agent: gotcha im really sure that this is cause a lot of trouble just then let me just pull up your account here and see whats going on so obviously you calling about the number ending in  6123 [[sys.number]] ',
    'Agent: hello welcome to cricket nation this is  jake [[sys.name]]  i how can i help you',
    'Agent: mean there is a card on file here ending in 4384 [[sys.number]] Customer: yes yeah the 40 dollar upgrade its  60 dollars [[sys.money]] ',
    'Agent: hi there welcome to cricket nation my name is  jim Michael [[sys.name]]  how may i help you today',
    'Agent: i just let me go ahead and check that for you what is the phone number that were going to work with the Customer: its aah area code  956 yearly 48 [[sys.number]] ',
    'Agent: thanks for calling cricket wireless my name is  moises [[sys.name]]  who do i have a plan sure youre speaking with',
    'Agent: thanks for calling cricket wireless my name is moises who do i have a plan sure youre speaking with Customer:  phyllis mercer [[sys.name]] ',
    'Agent: okay great no worries in that case just going to access in the account can i please have your phone number Customer:  8637380748 [[sys.number]] ',
    'Agent: okay perfect so when youre ready just give me the card number Customer: okay okay its  42322302 [[sys.money]] ',
    'Agent: okay can i also have the name that appears on the card Customer:  phyllis mercer [[sys.name]] ',
    'Agent: awesome perfect so in this case phyllis ill just need you to give me one more time the car and Customer: okay its  423223027555946 [[sys.number]]  XXXXXXXXXXXXXXXX',
    'Agent: let me read that back thats  8637380748 [[sys.number]]  is that right',
    'Agent: thank you for choosing cricket this is  jack [[sys.name]]  speaking may i get your name',
    'Agent: thank you for choosing cricket this is jack speaking may i get your name Customer:  timothy [[sys.name]]  im doing well get',
    'Agent: all right so miss 8I totally understand ill definitely assist you with that today dont worry about it youre reading the service gonna check the towers at the coverage everything dont worry aah 1 of all can i get the phone number Customer: okay  706 [[sys.number]] ',
    'Agent: all right lemme see let me access your account right now no all right so im in the account right now to see aah lets go ahead and start and see 1 of all you got the  60 dollar [[sys.currency]]  plan aah all right since see can i get your address to check the coverage around you',
    'Agent: thank you for calling cricket my name is  danny [[sys.name]]  tell me how how can i help you',
    'Agent: oh okay i am deeply sorry to hear about your phone aah being like damage but ill be more than glad to assist you today and help you file your app insurance claim cause i know for a fact that its a really important for you to have a phone working all the time now can i get the telephone number that is attached to the aah damaged phone please Customer: 502 aah 718 let me make sure im glad  467 in 4 [[sys.number]]  mm hmm',
    'Agent: thank you  4674 [[sys.number]]  all right i might have your account here in front of me but in order for me to access and check your account information aah can i get your name maam for my notes',
    'Agent: okay nice to meet you in the call and in the meantime i just wanted to say thank you because based on my computer you open up there account on 2015 thank you so very much for choosing this company now just going back into the main reason of your call i see that you have a aah family plan with 3 numbers and you said that youre having this issue with the aah phone that you have on the line ending in  4674 [[sys.number]]  right you said that the the aah screen got cracked on the device right',
    'Agent: thank you for calling cricket this is  zach [[sys.name]]  speaking may i get your name please',
    'Agent: yes please yes Customer:  5547370 [[sys.number]]  thats 1 of the numbers so families plan'
])


actual = ['slot – Customer name',
        'slot – Current bill plan',
        'slot – New bill plan',
        'slot- credit/debit card number',
        'slot- Due date',
        'slot – Phone number',
        'slot – Phone number',
        'slot- Agent name',
        'slot – New bill plan',
        'slot- Agent name',
        'slot – Phone number',
        'slot- Agent name',
        'slot – Customer name',
        'slot – Phone number',
        'slot- credit/debit card number',
        'slot – Customer name',
        'slot- credit/debit card number',
        'slot – Phone number',
        'slot- Agent name',
        'slot – Customer name',
        'slot – Phone number',
        'slot – Current bill plan',
        'slot- Agent name',
        'slot – Phone number',
        'slot – Phone number',
        'slot – Phone number',
        'slot- Agent name',
        'slot – Phone number'
]

f1 = f1_score(actual, preds, average='macro')

acc = accuracy_score(actual, preds)

print([runtime, f1, acc])

`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/30 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1260
  Num epochs = 3
  Total optimization steps = 315
  Total train batch size = 12
running astray


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/105 [00:00<?, ?it/s]

Iteration:   0%|          | 0/105 [00:00<?, ?it/s]

Iteration:   0%|          | 0/105 [00:00<?, ?it/s]

[33.616238344, 0.5660597572362278, 0.75]


In [2]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import torch
import pandas as pd

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

model = SetFitModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

train_ds = pd.read_csv("/home/ubuntu/nilay/Datasets/cricket_final_train_df.csv")
test_ds = pd.read_csv("/home/ubuntu/nilay/Datasets/processed_df_08-16-2021_13-21-00_sid_1675508193_dbsid_504.csv")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)


trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=12,
    column_mapping={'Class':'label', 'Text':'text'},
    num_iterations=30, # Number of text pairs to generate for contrastive learning
    num_epochs=3 # Number of epochs to use for contrastive learning
)

start = time.process_time()
trainer.train()
runtime = time.process_time() - start

preds = trainer.model.predict([
    "agent: so were ready to ready to make the payment mister [P] christopher [[sys.person]][/P] 12 minutes the name on the card",
    "agent: uh just yes its gonna give you the sign up for autopay after number no no [P] bridgeville [[sys.gpe]][/P] remove number on the rest of your week and i made you cannot make any type of changes",
    "agent: else that i can help you with today sir thank thank you [P] christopher physician [[sys.person]][/P] cricket my name is ronald you have a great day okay",
    "agent: let me go ahead and get everything set up set up again i wouldnt be regarding account numbers customer: [P] 442333500004474004 [[sys.cardinal]][/P]",
    "agent: got it will be like would be the card number customer: [P] 4423233535000474004 [[sys.cardinal]][/P] for",
    "agent: okay so just remember your basically it doesnt give you doesnt give you more or less of [P] 7 [[sys.cardinal]][/P] days okay its exact exact 7 days and then going to give you the option to make your second payment can payments so just to make sure you make a payment on it and all that kind of time if you do not make a payment on the 23 enter your bill cycle on your account is gonna reached restart and then the system will be asking you for you for the full amount to restore your services",
    "agent: zip code number customer: [P] 71241 [[sys.cardinal]][/P]",
    "agent: sure i can definitely help you with help you with that is that information on your account may i have your phone number please customer: 18 or 9 or [P] 978550 [[sys.cardinal]][/P]",
    "agent: okay well the expiration date customer: [P] 7/7/20 [[sys.date]][/P] been 24",
    "agent: [P] 0004 [[sys.cardinal]][/P]",
    "agent: sure i can definitely help you with help you with that is that information on your account may i have your phone number please customer: [P] 18 [[sys.cardinal]][/P] or 9 or 978550",
    "agent: on file until the [P] 23 [[sys.cardinal]][/P] to make your second payment is that okay is that okay",
    "agent: okay so just remember your basically it doesnt give you doesnt give you more or less of 7 days okay its exact exact [P] 7 [[sys.cardinal]][/P] days and then going to give you the option to make your second payment can payments so just to make sure you make a payment on it and all that kind of time if you do not make a payment on the 23 enter your bill cycle on your account is gonna reached restart and then the system will be asking you for you for the full amount to restore your services",
    "agent: so were ready to ready to make the payment mister christopher [P] 12 [[sys.cardinal]][/P] minutes the name on the card",
    "agent: okay so just remember your basically it doesnt give you doesnt give you more or less of [P] 7 days [[sys.duration]][/P] okay its exact exact 7 days and then going to give you the option to make your second payment can payments so just to make sure you make a payment on it and all that kind of time if you do not make a payment on the 23 enter your bill cycle on your account is gonna reached restart and then the system will be asking you for you for the full amount to restore your services",
    "agent: sure i can definitely help you with help you with that is that information on your account may i have your phone number please customer: 18 or [P] 9 [[sys.cardinal]][/P] or 978550",
    "agent: so like i know you have on the [P] 233 [[sys.cardinal]][/P] to make a second payment in this gonna be off of $53 sir so just remember remember that today we set up for a bridge pay every process the payment is a payment of $15 is there anything else thing else that i can help you with today",
    "agent: no im asking you the last 4 digits digits a [P] 0004 [[sys.cardinal]][/P]",
    "agent: so were ready to ready to make the payment mister christopher [P] 12 minutes [[sys.duration]][/P] the name on the card",
    "agent: so like i know you have on the 233 to make a second payment in this gonna be off of $53 sir so just remember remember that today we set up for a bridge pay every process the payment is a payment of [P] $15 [[sys.money]][/P] is there anything else thing else that i can help you with today",
    "agent: hmm i think i got a wrong number or the credit the credit for the card number so can you give it to me one more time please time please customer: [P] 44233500047460004 [[sys.cardinal]][/P] for",
    "agent: okay so just remember your basically it doesnt give you doesnt give you more or less of 7 days okay its exact exact [P] 7 days [[sys.duration]][/P] and then going to give you the option to make your second payment can payments so just to make sure you make a payment on it and all that kind of time if you do not make a payment on the 23 enter your bill cycle on your account is gonna reached restart and then the system will be asking you for you for the full amount to restore your services",
    "agent: thank you sir well the expiration version date on the card customer: [P] 0 [[sys.cardinal]][/P] row sim 24",
    "agent: okay so just remember your basically it doesnt give you doesnt give you more or less of 7 days okay its exact exact 7 days and then going to give you the option to make your second payment can payments so just to make sure you make a payment on it and all that kind of time if you do not make a payment on the [P] 23 [[sys.cardinal]][/P] enter your bill cycle on your account is gonna reached restart and then the system will be asking you for you for the full amount to restore your services",
    "agent: zip zip code number customer: [P] 7 [[sys.cardinal]][/P] in 12 for more",
    "agent: okay great so im in your account sir to set up the best thing you would just have to make a payment right now right now of [P] $15 [[sys.money]][/P] and you will have",
    "agent: no im asking you the last [P] 4 [[sys.cardinal]][/P] digits digits a 0004",
    "agent: okay well the expiration date customer: 7/7/20 been [P] 24 [[sys.cardinal]][/P]",
    "agent: [P] 304 [[sys.cardinal]][/P]",
    "agent: zip zip code number customer: 7 in [P] 12 [[sys.cardinal]][/P] for more",
    "agent: so like i know you have on the 233 to make a second payment in this gonna be off of [P] $53 [[sys.money]][/P] sir so just remember remember that today we set up for a bridge pay every process the payment is a payment of $15 is there anything else thing else that i can help you with today",
    "agent: for for the [P] 46 [[sys.cardinal]][/P]",
    "agent: thank you sir well the expiration version date on the card customer: 0 row sim [P] 24 [[sys.cardinal]][/P]",
    "agent: were gonna were gonna go get nation my name is [P] brian [[sys.person]][/P] amount of amount of that is because address of aah can i help you today sir",
    "agent: else that i can help you with today sir thank thank you christopher physician cricket my name is [P] ronald [[sys.person]][/P] you have a great day okay",
    "agent: security code customer: [P] 446 [[sys.cardinal]][/P]",
    "agent: got it security code customer: [P] 4446 [[sys.cardinal]][/P]",
    "agent: got it will be like would be the card number customer: [P] 4423233535000474004 [[sys.cardinal]][/P]",
    "agent: okay well the expiration date customer: [P] 7/7/2024 [[sys.date]][/P]",
    "agent: thank you sir well the expiration version date on the card customer: [P] 7/7/2024 [[sys.date]][/P]"
])


actual = [4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    1,
    1,
    1,
    1,
    2,
    3,
    3
]

f1 = f1_score(actual, preds, average='macro')

acc = accuracy_score(actual, preds)

print([runtime, f1, acc])

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/30 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 4140
  Num epochs = 3
  Total optimization steps = 1035
  Total train batch size = 12
running astray


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/345 [00:00<?, ?it/s]

Iteration:   0%|          | 0/345 [00:00<?, ?it/s]

Iteration:   0%|          | 0/345 [00:00<?, ?it/s]

In [2]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import torch
import pandas as pd

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

model = SetFitModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

train_ds = pd.read_csv("/home/ubuntu/nilay/Datasets/cricket_final_train_df.csv")
test_ds = pd.read_csv("/home/ubuntu/nilay/Datasets/processed_df_08-16-2021_13-21-00_sid_1675508193_dbsid_504.csv")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)


trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=12,
    column_mapping={'Class':'label', 'Text':'text'},
    num_iterations=30, # Number of text pairs to generate for contrastive learning
    num_epochs=3 # Number of epochs to use for contrastive learning
)

start = time.process_time()
trainer.train()
runtime = time.process_time() - start

preds = trainer.model.predict([
    "agent: so were ready to ready to make the payment mister [P] christopher [[sys.person]][/P] 12 minutes the name on the card",
    "agent: uh just yes its gonna give you the sign up for autopay after number no no [P] bridgeville [[sys.gpe]][/P] remove number on the rest of your week and i made you cannot make any type of changes",
    "agent: else that i can help you with today sir thank thank you [P] christopher physician [[sys.person]][/P] cricket my name is ronald you have a great day okay",
    "agent: let me go ahead and get everything set up set up again i wouldnt be regarding account numbers customer: [P] 442333500004474004 [[sys.cardinal]][/P]",
    "agent: got it will be like would be the card number customer: [P] 4423233535000474004 [[sys.cardinal]][/P] for",
    "agent: okay so just remember your basically it doesnt give you doesnt give you more or less of [P] 7 [[sys.cardinal]][/P] days okay its exact exact 7 days and then going to give you the option to make your second payment can payments so just to make sure you make a payment on it and all that kind of time if you do not make a payment on the 23 enter your bill cycle on your account is gonna reached restart and then the system will be asking you for you for the full amount to restore your services",
    "agent: zip code number customer: [P] 71241 [[sys.cardinal]][/P]",
    "agent: sure i can definitely help you with help you with that is that information on your account may i have your phone number please customer: 18 or 9 or [P] 978550 [[sys.cardinal]][/P]",
    "agent: okay well the expiration date customer: [P] 7/7/20 [[sys.date]][/P] been 24",
    "agent: [P] 0004 [[sys.cardinal]][/P]",
    "agent: sure i can definitely help you with help you with that is that information on your account may i have your phone number please customer: [P] 18 [[sys.cardinal]][/P] or 9 or 978550",
    "agent: on file until the [P] 23 [[sys.cardinal]][/P] to make your second payment is that okay is that okay",
    "agent: okay so just remember your basically it doesnt give you doesnt give you more or less of 7 days okay its exact exact [P] 7 [[sys.cardinal]][/P] days and then going to give you the option to make your second payment can payments so just to make sure you make a payment on it and all that kind of time if you do not make a payment on the 23 enter your bill cycle on your account is gonna reached restart and then the system will be asking you for you for the full amount to restore your services",
    "agent: so were ready to ready to make the payment mister christopher [P] 12 [[sys.cardinal]][/P] minutes the name on the card",
    "agent: okay so just remember your basically it doesnt give you doesnt give you more or less of [P] 7 days [[sys.duration]][/P] okay its exact exact 7 days and then going to give you the option to make your second payment can payments so just to make sure you make a payment on it and all that kind of time if you do not make a payment on the 23 enter your bill cycle on your account is gonna reached restart and then the system will be asking you for you for the full amount to restore your services",
    "agent: sure i can definitely help you with help you with that is that information on your account may i have your phone number please customer: 18 or [P] 9 [[sys.cardinal]][/P] or 978550",
    "agent: so like i know you have on the [P] 233 [[sys.cardinal]][/P] to make a second payment in this gonna be off of $53 sir so just remember remember that today we set up for a bridge pay every process the payment is a payment of $15 is there anything else thing else that i can help you with today",
    "agent: no im asking you the last 4 digits digits a [P] 0004 [[sys.cardinal]][/P]",
    "agent: so were ready to ready to make the payment mister christopher [P] 12 minutes [[sys.duration]][/P] the name on the card",
    "agent: so like i know you have on the 233 to make a second payment in this gonna be off of $53 sir so just remember remember that today we set up for a bridge pay every process the payment is a payment of [P] $15 [[sys.money]][/P] is there anything else thing else that i can help you with today",
    "agent: hmm i think i got a wrong number or the credit the credit for the card number so can you give it to me one more time please time please customer: [P] 44233500047460004 [[sys.cardinal]][/P] for",
    "agent: okay so just remember your basically it doesnt give you doesnt give you more or less of 7 days okay its exact exact [P] 7 days [[sys.duration]][/P] and then going to give you the option to make your second payment can payments so just to make sure you make a payment on it and all that kind of time if you do not make a payment on the 23 enter your bill cycle on your account is gonna reached restart and then the system will be asking you for you for the full amount to restore your services",
    "agent: thank you sir well the expiration version date on the card customer: [P] 0 [[sys.cardinal]][/P] row sim 24",
    "agent: okay so just remember your basically it doesnt give you doesnt give you more or less of 7 days okay its exact exact 7 days and then going to give you the option to make your second payment can payments so just to make sure you make a payment on it and all that kind of time if you do not make a payment on the [P] 23 [[sys.cardinal]][/P] enter your bill cycle on your account is gonna reached restart and then the system will be asking you for you for the full amount to restore your services",
    "agent: zip zip code number customer: [P] 7 [[sys.cardinal]][/P] in 12 for more",
    "agent: okay great so im in your account sir to set up the best thing you would just have to make a payment right now right now of [P] $15 [[sys.money]][/P] and you will have",
    "agent: no im asking you the last [P] 4 [[sys.cardinal]][/P] digits digits a 0004",
    "agent: okay well the expiration date customer: 7/7/20 been [P] 24 [[sys.cardinal]][/P]",
    "agent: [P] 304 [[sys.cardinal]][/P]",
    "agent: zip zip code number customer: 7 in [P] 12 [[sys.cardinal]][/P] for more",
    "agent: so like i know you have on the 233 to make a second payment in this gonna be off of [P] $53 [[sys.money]][/P] sir so just remember remember that today we set up for a bridge pay every process the payment is a payment of $15 is there anything else thing else that i can help you with today",
    "agent: for for the [P] 46 [[sys.cardinal]][/P]",
    "agent: thank you sir well the expiration version date on the card customer: 0 row sim [P] 24 [[sys.cardinal]][/P]",
    "agent: were gonna were gonna go get nation my name is [P] brian [[sys.person]][/P] amount of amount of that is because address of aah can i help you today sir",
    "agent: else that i can help you with today sir thank thank you christopher physician cricket my name is [P] ronald [[sys.person]][/P] you have a great day okay",
    "agent: security code customer: [P] 446 [[sys.cardinal]][/P]",
    "agent: got it security code customer: [P] 4446 [[sys.cardinal]][/P]",
    "agent: got it will be like would be the card number customer: [P] 4423233535000474004 [[sys.cardinal]][/P]",
    "agent: okay well the expiration date customer: [P] 7/7/2024 [[sys.date]][/P]",
    "agent: thank you sir well the expiration version date on the card customer: [P] 7/7/2024 [[sys.date]][/P]"
])


actual = [4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    1,
    1,
    1,
    1,
    2,
    3,
    3
]

f1 = f1_score(actual, preds, average='macro')

acc = accuracy_score(actual, preds)

print([runtime, f1, acc])

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/30 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1500
  Num epochs = 3
  Total optimization steps = 375
  Total train batch size = 12
running astray


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

[8938.570435470001, 0.4093333333333334, 0.55]


In [2]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score

import pandas as pd

model = SetFitModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_trainset_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_testset_mod.xlsx")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)


trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=12,
    column_mapping={'Class':'label', 'Text':'text'},
    num_iterations=50, # Number of text pairs to generate for contrastive learning
    num_epochs=5 # Number of epochs to use for contrastive learning
)
start = time.process_time()
trainer.train()
runtime = time.process_time() - start

preds = trainer.model.predict([
    'Agent: youre more than welcome whats your name again alright  richard [[sys.name]]  thank you so much alright so i',
    'Agent: unlimited okay okay so in this case did you want to change the plan now or you want us to process the payment for the regular  40 dollars [[sys.money]] ',
    'Agent: hello welcome to cricket nation this is jake i how can i help you Customer: hi i was calling to do the payment in 2 im upgrading my my services well actually just to a higher my services to little bit more expensive for now much 30 dollars so i wanna  40 dollars [[sys.money]] ',
    'Agent: okay perfect im ready to process the payment may i please have the card number Customer: okay it is  4491710136 [[sys.number]] ',
    'Agent: because you have 3 lines on the 55 dollar plan your due date was actually yesterday but  11/50/9 PM [[sys.time]]  no this is',
    'Agent: okay so just inform them to re submit the order okay because that is the information that i see right now for the cellphone phone number aah ending again  1471 [[sys.number]]  there is the port protection so provide them again the aah aah aah port out number to port ahead to call this one the transfer pin and they will have to aah started over and reset make the order and i usually that should really work',
    'Agent: gotcha im really sure that this is cause a lot of trouble just then let me just pull up your account here and see whats going on so obviously you calling about the number ending in  6123 [[sys.number]] ',
    'Agent: hello welcome to cricket nation this is  jake [[sys.name]]  i how can i help you',
    'Agent: mean there is a card on file here ending in 4384 [[sys.number]] Customer: yes yeah the 40 dollar upgrade its  60 dollars [[sys.money]] ',
    'Agent: hi there welcome to cricket nation my name is  jim Michael [[sys.name]]  how may i help you today',
    'Agent: i just let me go ahead and check that for you what is the phone number that were going to work with the Customer: its aah area code  956 yearly 48 [[sys.number]] ',
    'Agent: thanks for calling cricket wireless my name is  moises [[sys.name]]  who do i have a plan sure youre speaking with',
    'Agent: thanks for calling cricket wireless my name is moises who do i have a plan sure youre speaking with Customer:  phyllis mercer [[sys.name]] ',
    'Agent: okay great no worries in that case just going to access in the account can i please have your phone number Customer:  8637380748 [[sys.number]] ',
    'Agent: okay perfect so when youre ready just give me the card number Customer: okay okay its  42322302 [[sys.money]] ',
    'Agent: okay can i also have the name that appears on the card Customer:  phyllis mercer [[sys.name]] ',
    'Agent: awesome perfect so in this case phyllis ill just need you to give me one more time the car and Customer: okay its  423223027555946 [[sys.number]]  XXXXXXXXXXXXXXXX',
    'Agent: let me read that back thats  8637380748 [[sys.number]]  is that right',
    'Agent: thank you for choosing cricket this is  jack [[sys.name]]  speaking may i get your name',
    'Agent: thank you for choosing cricket this is jack speaking may i get your name Customer:  timothy [[sys.name]]  im doing well get',
    'Agent: all right so miss 8I totally understand ill definitely assist you with that today dont worry about it youre reading the service gonna check the towers at the coverage everything dont worry aah 1 of all can i get the phone number Customer: okay  706 [[sys.number]] ',
    'Agent: all right lemme see let me access your account right now no all right so im in the account right now to see aah lets go ahead and start and see 1 of all you got the  60 dollar [[sys.currency]]  plan aah all right since see can i get your address to check the coverage around you',
    'Agent: thank you for calling cricket my name is  danny [[sys.name]]  tell me how how can i help you',
    'Agent: oh okay i am deeply sorry to hear about your phone aah being like damage but ill be more than glad to assist you today and help you file your app insurance claim cause i know for a fact that its a really important for you to have a phone working all the time now can i get the telephone number that is attached to the aah damaged phone please Customer: 502 aah 718 let me make sure im glad  467 in 4 [[sys.number]]  mm hmm',
    'Agent: thank you  4674 [[sys.number]]  all right i might have your account here in front of me but in order for me to access and check your account information aah can i get your name maam for my notes',
    'Agent: okay nice to meet you in the call and in the meantime i just wanted to say thank you because based on my computer you open up there account on 2015 thank you so very much for choosing this company now just going back into the main reason of your call i see that you have a aah family plan with 3 numbers and you said that youre having this issue with the aah phone that you have on the line ending in  4674 [[sys.number]]  right you said that the the aah screen got cracked on the device right',
    'Agent: thank you for calling cricket this is  zach [[sys.name]]  speaking may i get your name please',
    'Agent: yes please yes Customer:  5547370 [[sys.number]]  thats 1 of the numbers so families plan'
])


actual = ['slot – Customer name',
        'slot – Current bill plan',
        'slot – New bill plan',
        'slot- credit/debit card number',
        'slot- Due date',
        'slot – Phone number',
        'slot – Phone number',
        'slot- Agent name',
        'slot – New bill plan',
        'slot- Agent name',
        'slot – Phone number',
        'slot- Agent name',
        'slot – Customer name',
        'slot – Phone number',
        'slot- credit/debit card number',
        'slot – Customer name',
        'slot- credit/debit card number',
        'slot – Phone number',
        'slot- Agent name',
        'slot – Customer name',
        'slot – Phone number',
        'slot – Current bill plan',
        'slot- Agent name',
        'slot – Phone number',
        'slot – Phone number',
        'slot – Phone number',
        'slot- Agent name',
        'slot – Phone number'
]

f1 = f1_score(actual, preds, average='macro')

acc = accuracy_score(actual, preds)

print([runtime, f1, acc])

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/50 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 2100
  Num epochs = 5
  Total optimization steps = 875
  Total train batch size = 12
running astray


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/175 [00:00<?, ?it/s]

Iteration:   0%|          | 0/175 [00:00<?, ?it/s]

Iteration:   0%|          | 0/175 [00:00<?, ?it/s]

Iteration:   0%|          | 0/175 [00:00<?, ?it/s]

Iteration:   0%|          | 0/175 [00:00<?, ?it/s]

[89.86498472299999, 0.5566580478345184, 0.75]


In [3]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score

import pandas as pd

model = SetFitModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_test_shot.xlsx")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)


trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=12,
    column_mapping={'Class':'label', 'Text':'text'},
    num_iterations=30, # Number of text pairs to generate for contrastive learning
    num_epochs=3 # Number of epochs to use for contrastive learning
)
start = time.process_time()
trainer.train()
runtime = time.process_time() - start

preds = trainer.model.predict([
    "hello hi welcome to cricket mission minimum my name is jimmy how may i have your name please",
    "that's still i'm still a was able to clear it to you uh thank you for thank you for calling",
    "welcome to the ticket to take your permission my name is add may i have your name please please",
    "all right well thank you very much for calling through good i'm good good i'm glad i was able to add a hot spot and of and uh basically answer the question about the bill so this is so this is now once again thank you very much for being the best part of cricket you do have a great day bye for now",
    "welcome to cricket click admission my name is john can i have your name please name please",
    "hi you're welcome hi welcome to cricket things on this is josie may i have your i have your name please",
    "okay i'm very happy with i was able to resolve your concern by canceling one last two line two lines on the account this is laurel again and thank you for choosing crickets tuner you have a wonderful day and stay safe",
    "hi thank you for choosing tricking tricky my name is daniel with them speaking today",
    "welcome to cricket assist this is john may i have your name please",
    "hello welcome to cricket nation my name is my name is john how can i help you today",
    "hello welcome to cricket my name is jay jade may i have your name please hi there welcome",
    "hello welcome to cricket ma'am how can i help you can i help you",
    "hi welcome to click in my name is jay may i have your may i have your name please",
    "thank you so much i'm glad to i'm glad i was able to resolve your concern and by the way by the way i my name is an aerial when that thank you so thank you so much for calling i wish you",
    "okay aah so so yeah i guess it once the patiently for every time i'm happy happy with able to assist you fifty and so i'm sorry so thank you so much for calling cricket have it for you have a great day",
    "hello welcome to welcome to cricket my name is anna may i please have your name",
    "welcome to create an agent major in my name is daniel with whom i have the pleasure to speak with take this",
    "welcome to cricket cricketing tion my name is rafi l and my hair in i have your name please",
    "thank you for calling verizon wireless my name yes my name is better which one will i be having a pleasure let's you're speaking with",
    "hello hello welcome to cricket this is hazel how can i help you you",
    "you're welcome to cricket to cricket this has been nonna may i have your name please",
    "all right you're welcome ma'am please thank you so much for calling have a great day stay safe",
    "hi welcome to tickets nation my name is my name is berna how can i help you today",
    "welcome to create condition mister here how can i help you today",
    "yes sir well actually well actually the payments is actually prester right now now what i recommend sir is to turn off turn off the phone and turn it back on and everything should be working you're welcome sir",
    "welcome welcome to coordination this is jonah can i have your name have your name",
    "welcome to cricketing coordination this is josie may i have your name please",
    "good evening welcome to cricket my name is name is danielle how may i have the pleasure to speak to",
    "good evening good evening you mean welcome to cricket my name is danielle how may i have i have the pleasure to speak to",
    "hmm hi robert how you doing today",
    "also high hi how can i help you",
    "hi there very welcome to the technician is the one let me have your name your name please",
    "hi gary how can i help you today help you today",
    "if you upgrade as per as per checking here you're eligible for a device that hi discount so that means if you plan you plan on upgrading your device you can definitely and i only have a discount as long as you're logged log in your account over cricket wireless dot com com you can choose your device will recover device device if you should be able to receive a discount discount",
    "hi good morning this is this is chris from cricket how can i help you today okay",
    "hi does how does right how can i help you",
    "hi welcome to coordination this is christian christian can i have your name please",
    "hi hi welcome to the technician this is using sure we can have your first name please",
    "alright hi sir thank you so much thank you so much for patiently waiting um yeah i was able to check here check here aah i really would get an get an error it shows device not um if you don't mind if you don't mind me asking 'cause this phone is uh i'd be way older phone meaning it's it's a non ticket phone ticket phone okay now aah before you for it was activated to cricket where it was activated to read it before",
    "okay well let's pull up your pull up your account first together aah are you calling from calling from el a phone number ending seven eight eight eight eight zero",
    "alright let me so happy i was able to work with your account in your account today and help you out with your options again this is hazel thank you for your time calling cricket cricket and have a nice day okay",
    "thank you very much you're very much for calling to coordination that i made a speaking we can you please primarly for your name",
    "perfect sir so thank you so thank you so much for calling cricket my",
    "perfect sir so have a good day sir thank you so much for calling trigger",
    "that's good to hear by the way can by the way can i have your phone number that you're calling about about please",
    "hello thank you for calling cregan wireless is so since all extenders speaking how can i assist you today",
    "yeah okay arroba don't worry don't worry i got to be more than glad to assist you and let me let me take a look here and take care about your information information may i have the phone number associated with your with your account please",
    "okay well my so well my goal today was to assist you added the the uh rate plan for a much better better one and that way that you can get the the mobile hot spot in your two services like this line then i go was this so still help you out today and referral and thank you for choosing which is in cricket by a services provider that number remember my",
    "hello hello april how can i help you today",
    "hmm sorry to hear that your internet as well this off no worries i can help you with that so so before we proceed are we working on the number and number and in two nine six seven",
    "or you have the wifi on your help on how on your house",
    "miss green that us how can we help you",
    "oh okay i do i do apologize to hear then and don't worry i can help you may i help you with that let's go ahead and take a look take a look can i have your phone number mother",
    "thank you thank you very much for that information okay okay let me go ahead and your help you help you try to unlock the device here and just hold on hold on just let me open the total tried online online that device as well yeah hold on",
    "okay sure and can i help you out can i help you out in the hospital i can give me your name",
    "perfect sir so any notes here to help today",
    "awesome thank you and how can i help you by the way by the way",
    "hmm sure megan and i can help you with that disconnect one of your line can i have the have the number that you would like to uh removing remover",
    "hi welcome to click to cricket my name is jay may i have your name please",
    "hello hi hi hi this is this is jay from cricket service actually on the phone phone with rochelle",
    "all righty well thank you very much for being for being the best part of cricketing in my name is jay they do you have a great one okay stay safe",
    "welcome welcome to cricket my name is jay may i have your name have your name please",
    "name is jay and have a great day goodbye for now",
    "hi welcome to click in my name is jay may i have your may i have your name please",
    "alright well thank you very much much for being the best part of cricket again my name is jay have a great one today and stay safe okay",
    "okay this is jay getting have your name your name",
    "hi welcome to to cricket mention this is jay can i have your name",
    "you're welcome and thank you so much for much for calling cricket this is jay you do have a wonderful day",
    "okay aah currently currently you both have the thirty dollar hello plan if he switching of fraud aah from thirty to sixty your your monthly will be ninety dollars",
    "hello",
    "hello hello",
    "thank you um have um hello there can it thank you so",
    "aah hello thank you so much for so much for patiently waiting um let me just",
    "hello detail yes",
    "hello hello thing as i'm not sure you're calling freakin' wirless this is joe it says y'all speaking how can i help you today",
    "hello sir thank you for patiently for patiently waiting on the line does a check check on the account here and and uh i'm checking here there isn't as actually a payment that was made on the thirteenth fifteenth but what i can see here is that there was a there was actually one attempt of thing of a payment payment here last twelve of of uh august and um you hmm you made the payment online i'm i correct",
    "hello welcome hello welcome to cricket my name is jen how can i help you",
    "hello welcome to welcome to try to wireless my name is jennifer may i have your name please and please",
    "laurie alright thank you aah if you can go back to your my cricket app app",
    "aah aah okay normally would cricket if you'd if you're going to be making an upgrade of of the other line we always recommend that you do the and do that for both lines so that we can take advantage take advantage of a bundle discount it it's cheaper",
    "aah two options that they can they can separate a number lorraine it's either either you can aah go to the cricket store get store the store can actually do that within the day day or we can also file file a ticket to back office back office signal but the one to call you and set and separate the numbers over the phone but but the ticket usually will take three to five business days space for a call back",
    "aah because you only have i only have paid for partial of the account account remember you have activated the and the second line just july twenty nine nine supposedly your bill is sixty sixty dollars or sixty eight i would say because there's 'cause there's cricket insurance on the other number but you only paid forty eight it's because it's because you only paying for the proration tion of those days left on the billing side billing cycle so supposedly simply aah you you paid up for for the service of the phone number ending ending four one nine six from july july twenty nine it's activation up until aah august six aah like aah august seventeen when we start and restart your new billing cycle",
    "means hmm we cannot unless unlock this device because it is only been active activated for three days with a is a cricket sam",
    "thank you so much and i'm glad i was and i was able to resolve your concern by uh now so that you can now enjoy the retail services once once again nicole this is berna and thank you purchasing cricket and have a great day bye",
    "okay and if and just gonna go ahead and check something because it seems that this phone this phone is not from cricket",
    "yeah yeah the device it doesn't displace any and he called because it's not frank frank cricket but the other thing that you need in order for you to move out to okay so another deploying other provide the provider it's your account number and the but in the password or pin mother",
    "by by the way for me to helping you out from cricket",
    "hmm cricket is a prepaid it's a prepaid service uh robert there's no refund refund we don't uh credits either the customer aah if if the if they pay the service to get a use up until until uh the next few days because again once and once we",
    "okay and that is for which aah uh which rate plan aah which phone number number the four one nine six",
    "will be the start of your new billing cycle that one oh that would start of course august seventeen up up until september sixteen so for so for example if indeed then at the middle of the billing cycle in cycle you'd wanna separate the number number they will going to prorate those days was days aah left on your billing cycle cycle as a refund and then the other line the airline uh that will be separated will be created a brand new aah bill aah billing cycle it's own billing",
    "of you will get that twenty five dollars hours with uh refer offend aah credit but if it's added on your account your account it's not a uh before for it verifying",
    "purchase by just having the i m e a april i need to pull up an account account for uh as well your",
    "all right all right still looking in here uh april just hold on hold on",
    "um april was able to able to pull up an account april oscar check upon checking here this device was activated dated on november seventh seven and has been deactivated on of them november nine and the last the lines of service is uh of this device why is this just three days so that",
    "hmm just give me a second here i will and i will just check hmm okay uh uh i'm checking here you know that our on your area yeah yeah but all the tower is working fine and active active so let us go ahead and put and then proceed with the troubleshooting of your device so did you try any trouble and the trouble shooting on your device before",
    "yeah that's why uh is it okay if you can check can check if your cellular data is turned stand on",
    "uh on the settings thanks and then you have to select cellular",
    "uh you should turn on that one",
    "hi hi welcome to cricket my name is judy how how can i help",
    "hi welcome tickler destination that is area how so how can i help you today",
    "hi welcome to welcome to cricket nation my name is kingston may ask may i ask for your name please",
    "hi welcome to cricket you should this is allen speaking may i know your name okay",
    "hi welcome to cricket nation my name is linda and then may i have your name",
    "hi welcome to click on save uh thank you so much for patiently waiting on the line dan i do apologize apologize columb wait all right all right so i just refresh the cricket account here here and i've just made made sure that the phone number ending ending in five one one four is is associated with the iphone x r are let me double check every so it's reloading one more one more time so we can make sure it definitely make sure that make sure that this will not be suspended again okay",
    "hi welcome to recognition sir my name is raffia may i have your name please can you hear me hear me okay",
    "hi welcome to cricket my name is name is one c may i have your name please",
    "hi welcome to cricket mission patient is john how can i help",
    "hi welcome to click to cricket my name is ryan may i have your name please",
    "alright john all right johnny i'm just waiting for a response to stay with me okay with me okay we're",
    "thank you john y",
    "um if you have a new device you have a do you have a device john because of course with she for us to activate it line we need to have a device",
    "well currently currently john we have uh you can get zero dollars zero dollar devices if you transfer from from another service provider to cricket",
    "welcome to cricket nation my name is john is john can i have your first and last name",
    "you're most welcome ma'am is your your services once again this has been john ma'am thank you for being the best part of cricket nation keep safe okay",
    "oh hi must john how may i help you today day",
    "cricket cricket mission my name is may john and your name please name please",
    "you don't use all that all the gigs that's make an example john use my use five gigs can will be missing any and it will be lost because it will not be like stuck they will be lost you lost",
    "hello good morning my new welcome to cricket nation my name is john john how can i help you today",
    "alright so brandon having that i was able to give you the breakdown of the charge the charges an option to separate the number i will note that in your account if in case you go to the store to the store they can pick up where we left off yes okay",
    "be i won't be able",
    "all right let me just check all right april for this one i this one i'm able to unlock there's an unlock code or hold are you ready do you have pen and paper with you",
    "unlocked not unlocked uni you it will be able it will be able to transfer it",
    "okay okay three five six one and just a disclaimer klamer robert if in case in the future okay okay if you would like to reactivate this line you only have only have twenty nine days you're going to pay whatever the whatever the plan price and fifteen dollars for the activation should be the one twenty nine days when able to read would you reactivate the same number but i know you can and get a new sim card and activate a new number instead instead okay",
    "okay and i'm glad i was able to resolve resolve the issue the phone number ending in three five six one six one completely disconnected or removed remove your updated balance from one from one eighty six it went down to one fifty six fifty six dollars uh with your autopay do they do on uh september eight okay and",
    "aah if you want let me see if we could frequent because supposedly there should be a bridge the see which page extension tension where you can just process at least an initial initial payment for example fifteen dollar initial michelle payments and then your services will be be good and l the twenty six we have that but but i need to further check your account if if you'd be able to um process any payment would you like to do bridge bridge bridgeport or would you like to really process the whole the whole amount",
    "able",
    "and there there you'll be able to aah click click on this and that'll be it to you uh you will have",
    "the pleasure to speak to",
    "be more than glad to glad to assist you with that um we do i have the do i have the pleasure to speak with",
    "welcome to the quick admission my name is edwin when please tell me who i'm glad to speak with today with today welcome to the combination they sure my name is edwin please tell me tell me where i'm glad to speak with today speak with today hello hello hello hello welcome to the green one that's mention this is that when that when you tell me how may i help you today",
    "amount of the night ninety dollars that you can also settle through the my through the my cricket app or through the cricket wireless dot com dot com and there you go run the congratulations gotcha lesions as we speak i have successfully successfully take care of it for you and it's now and unlimited plan i mean unlimited data for the for the fifty five dollar",
    "and because and because of the following four nineteen example i had one sent you situation i'm going to be direct with you i'm going to speak speak straight up to you i'm sorry i don't mean to be done by the one i don't want to be rude talkin' about this no no no no i'm on my",
    "thank you for calling thank you for calling quickening sure my name is is uh with one would wanna speak it right now",
    "and that's my screen broke on my so i put my sim card into an iphone iphone and it's saying that i put it is but it is not you can make a payment from i mean as being payments aah on um on friday but speak so so i'm not sure what it's saying you make a payment",
    "hello thank you so much thank you so much for calling crickets mention this is jacqueline how much do i have the pleasure to speak with",
    "hi welcome to cricket nation despaired speak speaking can i please have your name hi welcome to cricket mission my name is bert can i have your name please hello hello",
    "hi hi welcome to cricket nation this this is shane speaking speaking may i have your name please hello welcome to create because my son is this is shane speaking speaking may i have your name hello hello hello welcome to cricket mention this is shane speak speaking may i have your name hello hello aah aah memory sir if you could hear me i just wanted to it's inform you that i'm not hearing any response once from you hello hello hello ma'am are you sir if you could hear me again again i just want to inform you that i'm not hearing harry any response from you hello hello hello welcome to cricket nations this is james shane speaking may i have your name okay hello aah memory sir if we could she can you hear me again i'm not hearing and a response once you this this will be the devices might disconnect the call now to accommodate more customer customers reading on here she has to further to further questions about your account the service finish the start star the number again okay thank you",
    "tell me how can i ask you can i assist you today sir",
    "all right so i'm glad i was able to able to assist you regarding with this concern kennett kenneth and just found a reminder that the account account is still good to go up until on the twenty third of august and",
    "all right so john john high thing for settled here for today anything anything else i can further assist you with",
    "okay great great sir thanks for being part of the cricket family remember remember my name is edward was a pleasure to assist you this year and hope you get better of your allergies i just",
    "righty don't worry i'll be more than be more than happy to assist you with that today can i have your name and name and the phone number",
    "yes ma'am is there anything else i can now i can assist you with",
    "all righty miss and do this as well recheck technical side to make a payment and we also cancel the line cancel the line ending let's see fifty sixty three thirty four it my goal to assist you with to see what this today have a compass that",
    "okay i got to be more than glad to assist you and i'm doing fine thank you for asking this a referral raphaela and how your self",
    "hello welcome to break it my name is danielle how yes how can i ask you if you have the name",
    "uh hi welcome welcome to aah cricket frank speaking how can how can i help you",
    "hello welcome to cricket my name is yes my name is jen how can i help you hello this is jen from cricket how can i help you today help you today",
    "hi hi good morning welcome to cricket patient this issue this is your main how may i help",
    "thank you so much for calling crigger wireless my name is self itself and how can i help you today",
    "hi welcome to welcome to cricket mission my name is darcy may have your name have your name please",
    "uh my name is ernest was a pleasure to assist you thank you for calling the recognition",
    "welcome to welcome to",
    "hello welcome to coordination my name is roger roger with who i have the pleasure today",
    "hi hi thank you for calling cricket my name is bell how can i have your name please",
    "welcome to cricket to cricket my name is allison can i have your name",
    "welcome to clarification my name is twelve you're welcome have your name please",
    "all right you're welcome to verizon wireless the last one of the phone how can i assist you today",
    "welcome to cricket nation",
    "hi would welcome to coordination this is jessica how can i help you may i help you today",
    "welcome to bring",
    "hi welcome to clarification",
    "so i won't take too much of your time to ya yeah again thank you so much for calling cricket today good today my name is joan us once again you do have a great day and you take care as",
    "okay don't worry again thank you for calling verizon wireless have a great day",
    "hello welcome to correct to cricket this is my nonna can i get your name please please",
    "welcome welcome to the condition my name is i said it has a pleasure to speak to speak with doctor hello",
    "me too me so um for this one this one i can see your message jam that within within seven days seven business days you'll be able to receive an email l so let me just check check when was the um other line was active activate it",
    "thank you thank you so much miranda so let me give her let me go ahead and pull up your account here first alright alright so i got no and access of your account here aah let me check your mail hot spot aah",
    "thank you so much hold on much hold on alright thank you so much make much nichole for patiently waiting apologize for the hold",
    "all right thank you okay",
    "okay thank you you too goodbye",
    "okay i'll go over there i there i appreciate your time thank you",
    "right guess she still have it i don't know what she through the sim card the sim card away once you through the phone away she probably good thank you take it out so she went through a whole new carrier and i was and i was trying to place an instead of suspending it bring it",
    "too thank you then i have a wonderful day",
    "no you helped a lot thank you very much for much for your help",
    "too thank you bye bye",
    "no that'll be fine thank you so much",
    "so that's all thank you",
    "that's it that's the reason i'm just i mean i i've done all the all these years and i had a land base price this phone you know and i only a thirty minutes thirty minutes long this is of course he can't tax or anything or anything it went up to sixty bucks a month what were you guys are half of that and so that's that's you know you made me come to you and i should have i should've did this a long time ago but thank god thanks for anyway and i appreciate it and aah aah i'm gonna go here and one comments on it aah i can tell you care it's coming over and i really appreciate appreciate that i live by like that so thank you thank you",
    "all right uh alright awesome thank you so much",
    "okay thank you okay thank you so much",
    "uh uh thank you",
    "okay no problem thank you so much",
    "thank you so much have a blessed day",
    "okay sounds great thank you right thank you",
    "but again i appreciate you thank you",
    "i see okay i appreciate it appreciate it thank you so much for your help",
    "okay great okay thank you so much i much i really appreciate your help",
    "okay all right well i appreciate it hi everything you in pre check your help and i have and i hope you have a good rest of the day",
    "i appreciate you",
    "yes sir okay okay yes sir no appreciate it",
    "thank you sir i appreciate",
    "no that's all i needed appreciate you take your",
    "that i mean they were whenever i go get go get the the new phone i'm on i'm going to bring my laptop so they can so they can do i can do that then you know the other things things but i really appreciate it",
    "yeah but aah but anyway thank you so you know to ask you to do that to tell me i really appreciate it",
    "no that's all i wanted to know and i i know and i really appreciate your help and have a wonderful rest of the agent",
    "um my cell phone and then i it's a unlock i'm not not my my and then the i go to the do the um that is well i can use i can use the country the uh the sim card sim card so i'm good thank you so much for me let me",
    "depends on the phone okay hmm hmm okay sounds good thank you very much that's all i needed to pay okay",
    "you haven't you have a good thank",
    "see okay okay well good thank you so much i i really appreciate your time with more ticket have a great day",
    "okay very good thank you",
    "all right so sounds good thank you",
    "okay sounds good sounds good thank you",
    "okay sounds good thank you",
    "all right good thank you so much so much",
    "good thank",
    "thank",
    "okay okay i think i'll write it down now okay now okay thank you so much",
    "thank you hazel to hazel you as well",
    "okay okay all right thank you",
    "no thank you",
    "to thank you for helping me you have",
    "okay oh oh gotcha okay thank you it's hundred percent percent then let me take okay then you're calling to see if there's any kind of now this hot and he got it yes yes hundred i'm waiting on again",
    "thank you you too bye see i",
    "so that's all thank you",
    "thank the customer for their business aah send me a good morning or not i can i can let her i'm applying from cricket ticket",
    "all right thank you",
    "all right thank you very much",
    "okay okay so yeah i guess those details so much for everything all right thank you so much for calling frontier have a great day ahead bye for now",
    "okay i okay all right thank you so much and i'm",
    "all right thank you so much and you as",
    "okay all right okay all right thank you",
    "yeah it says aah is aah yeah uh bridge payment but which says uh right now right near from cricket your can setup bridge right thank you all right thank you for your bridge first page by payment you you need to make your bridge permanent by eight twenty eight twenty the by the twenty four",
    "all right thank",
    "okay sounds great thank you right thank you",
    "okay alright thank you all right thank you so much",
    "remember to sync the customer for their business uh april stoner",
    "it never had a phone number phone number i have there's another phone number that number they can't do anything uh i'm going to i'm turning on now you can't look it up at the serial number",
    "okay let's see uh uh settings harry um um ready um hmm about phone here we go it does not have a phone number it says unknown it gives me a model serial serial and i m e i number",
    "uh",
    "okay i okay i don't know when it says do i have another phone phone let me see um hold on uh all right right which was turning on all right settings so i have a serial number on not on this one",
    "uh i'm ready",
    "uh huh",
    "we're just saying the cost the customer for their business hi uh donna i will call i was calling because my internet is off",
    "it's saying it's saying it hasn't i i can't connect to it but do it for the past like uh five to ten minutes now now it says it down",
    "yeah connect the internet internet you're offline check your connection and and when i press crap um tap to uh um we uh uh try to connection or cellular cellular data connections are not available during the call what's goin' on with this phone can i can i get now no internet connection and i was trying to trying to go on my email",
    "okay thank you miss thank you to send you",
    "okay thank you",
    "okay thank you so turn off the phone for two minutes and turn it back on okay thank you thank you bye bye",
    "okay okay thank",
    "okay okay thank you",
    "okay thank you thank you",
    "okay thank you so much you so much",
    "okay thank you very much i appreciate it",
    "okay thank you so much i'll be careful with the thank you",
    "okay thank",
    "sixty eight dollars okay alright and so if i split the but this like i like i got this for someone for and for the birthday do i qualified for the twenty the twenty five dollar thing",
    "okay okay okay alright cool",
    "alright",
    "alright i just wanna it's one of two numbers aah because",
    "all right alright",
    "alright thanks sounds like something",
    "alright thank you got it thank you very much",
    "alright thank",
    "thank you also have to find that",
    "the app to absolutely not you've been so helpful thank you so much",
    "to thank you for helping me you have",
    "okay then well thank you so much",
    "okay that's what i needed to know it to know all right thank you very much much",
    "uh no no thank you",
    "very good thank you so much",
    "that's all thank you sir",
    "hmm hmm thank you",
    "no i'm good thank you for yeah i appreciate it",
    "so much higher i appreciate that",
    "of great thank you so much any",
    "thank you you too sir",
    "i'll call you back thank you",
    "um like i mean i don't think you thank you for that and she doesn't even charges for just phone i'm thinking that he might not needed i need it",
    "okay okay thank you so much for your help much for your help",
    "thank you very much much",
    "all right so sounds good thank you",
    "okay oh yeah so i got this this month my favorite alright okay came out",
    "okay one second and",
    "okay okay now supposed to one of us reflect needs more data than the other house how do i do that",
    "okay",
    "okay he he he wants to sixty dollars unlimited plan but",
    "six okay and then and then aah that'll be nice be ninety plus thirty and and if we if we both switched to the unlimited what what it what would that be",
    "okay so that's no deal for me i don't need i don't need unlimited okay now supposed like was like i i i got this line for real relative for the birthday",
    "okay okay and so what is it is there any cost for separating the ticket the lines",
    "okay so supposed like i said pirated read it today what would it cost",
    "okay so what do you mean less than thirty thirty",
    "okay so i don't",
    "so my next so if i so if i do not split it what's the next the next day the next bill what is going to be my next bill",
    "a great day",
    "great day",
    "a day before",
    "okay sir okay so for celisa something has to get a record of they told me to go comma falls companies get the phone the phone records or whatever i don't know how to i just need a i don't i can't see how to do that uh with uh i can't i can't i don't know how to look on your from what they said they don't is i'll go back that far it's no way i can go to somebody y'all can can print out the twenty fifth of the call them cause i made on that day",
    "i'm calling this is my third phone calls tell my aah it's twenty for some reason i'm not getting getting my uh internet can i know that that's then that's what this is the third call and this is too i two hours with the situation i don't think it's under just understand i'm kind of short on the page that i was told was gonna get paid and then it's internet is we're gonna call me back last thing would day was said make sure that all my oh my wifi aah you know i didn't know if then open rest of the distending and i still get very low bar far",
    "made my day they",
    "nope no i don't have no more i just i know you got going on how are you awesome mericans things around you're very much and you have a great day",
    "it doesn't help none and my best bet to you is this day this day and time brother you can be that healthy oh this or the port to healthiest my is my dad and the port my lives is this where i looked at it when you're time ago your time your time ago you know what i'm saying is",
    "or the day too to the whole",
    "nine alright and twenty three",
    "all right sure i can go ahead ahead and give you uh the information can so that will be able to know what you're going to do is to do if in case that you would like to suspend the line and filling up before i go ahead and do that let me ask you may i ask you how are you doing today",
    "hi how are you today ma'am hmm",
    "i am yeah i'm good ma'am how about you",
    "the door hi do you do you know how are you doing today",
    "and aah i would like to a steven for then for how long have you been using this device is active actively to cricket network",
    "thank you how's your day going so far going so far today miss shannon",
    "thank you and how and how are you doing today",
    "so your screen how long have you been using so new phone",
    "and i'm doing great thank you for asking how about you",
    "and how is your day going so far bye",
    "it's an a station high anastasia patient how is your day so far",
    "hi david good evening how are you today okay",
    "all right and how and how are you doing today with ya",
    "okay i thought i lost you aah i i hope you don't mind me asking be asking for and how are you doing today",
    "hmm how about you to try and send a text message and say",
    "lamont hyla month how are you doing today",
    "how are you are you",
    "uh t m c a nice submission t m how are you today ma'am today ma'am",
    "brian high ryan how are you doing how are you doing today",
    "very high area how are you doing how are you doing today",
    "way safe escalate would be the website and in case in case you go in and make a payment in the future you're doing you already know what to do okay",
    "hmm all right so what it as a what we will be the will be doing on the phone number ending on for one four one six nine four one nine six",
    "so yes i did yes i did how are you doing today",
    "thank you miss laura how are you doing today nice to have nice to have you on the line",
    "thank you ma'am thank you very good and how are you doing today",
    "all right and how and how are you doing today with ya",
    "hello how are you doing today mary anna anna",
    "it is if these other how are you doing today sir",
    "how are you doing today doing today salvatore's",
    "during high doreen how are you doing today",
    "very high area how are you doing how are you doing today",
    "how you doing today patricia",
    "aah aah okay normally would cricket if you'd if you're going to be making an upgrade of of the other line we always recommend that you do the and do that for both lines so that we can take advantage take advantage of a bundle discount it it's cheaper",
    "will be the start of your new billing cycle that one oh that would start of course august seventeen up up until september sixteen so for so for example if indeed then at the middle of the billing cycle in cycle you'd wanna separate the number number they will going to prorate those days was days aah left on your billing cycle cycle as a refund and then the other line the airline uh that will be separated will be created a brand new aah bill aah billing cycle it's own billing",
    "it's gonna be going to be six to eight dollars due on september number sixteen that's gonna be the regular bill yeah",
    "so are we going to be making the upgrade or not yet not yet",
    "well let me take a look at this one here here then give me just a moment let me check on check what's going on here aah the phone number you were referring to into is other one year on you're on right now ending in one two four five is that right is that right",
    "okay so aah now now i'll process first first the eight one six eight three two three five two seven hold on hold on and aah to confirm that you were going are we going to going to cancel this effective today",
    "foramen eight to reach you how are you today are you today",
    "everything everything's fine ma'am thank you very much for asking how are you there hours are they going",
    "hey cares how are you today",
    "hmm yes and then if you want to lose the device the device is the weather that are available that will that will show you ma'am",
    "all right all right it's seven seven eight let me know if you're okay just stay on the line aah and aah april all right so april how aah while i process it for you aah how's your day today",
    "by the way by the way i can send the system that the very front of you trying to do have needs to be updated into the fifty five fifty five unlimited core plan which is the same the same thing but the name is going to be updated say it and it needs to be updated for you to basically get a chance to apply to hi to program that we have in this case okay so you mentioned you broke up and if it problem if you would if you would like to applied we can send you the the device form for you to do it but it but that's it basically somethin' else um that you can do you know to have to have the when you get the may plan updated data but what i would need now is is to update the right so what do you think think mister michael because",
    "i'm good thank you so much for asking how are you doing are you doing today",
    "i'm great sir um um how about you",
    "hi baggage good afternoon the afternoon how are you today",
    "during high doreen how are you doing today",
    "morning thomas how are you today or today",
    "hello how are you doing today mary anna anna",
    "hi there clear how are you how are you",
    "i sorry to interrupt aah hi brandon hi brandon how are you doing today",
    "agent how are you today",
    "hello their return richard how are you",
    "how you doing today patricia",
    "hi i'm in the how are you",
    "i'm really good good thanks for asking how about you",
    "hi welcome to tickets nation my name is my name is berna how can i help you today",
    "hmm hi robert how you doing today",
    "also high hi how can i help you",
    "hi there very welcome to the technician is the one let me have your name your name please",
    "hi gary how can i help you today help you today",
    "if you upgrade as per as per checking here you're eligible for a device that hi discount so that means if you plan you plan on upgrading your device you can definitely and i only have a discount as long as you're logged log in your account over cricket wireless dot com com you can choose your device will recover device device if you should be able to receive a discount discount",
    "hi good morning this is this is chris from cricket how can i help you today okay",
    "hi does how does right how can i help you",
    "hi welcome to coordination this is christian christian can i have your name please",
    "hi hi welcome to the technician this is using sure we can have your first name please",
    "hmm just give me a second here i will and i will just check hmm okay uh uh i'm checking here you know that our on your area yeah yeah but all the tower is working fine and active active so let us go ahead and put and then proceed with the troubleshooting of your device so did you try any trouble and the trouble shooting on your device before",
    "okay okay three five six one and just a disclaimer klamer robert if in case in the future okay okay if you would like to reactivate this line you only have only have twenty nine days you're going to pay whatever the whatever the plan price and fifteen dollars for the activation should be the one twenty nine days when able to read would you reactivate the same number but i know you can and get a new sim card and activate a new number instead instead okay",
    "i know you're you're do is actually actually um okay you're you made you made uh the last payment successful was on the on july right",
    "with regard regards to that one we need to check also the area area and see what could have been the reason the reason why it's acting up that way but uh but uh for the payments you've made that that went towards your because you were in the sixty dollar sixty dollar plan before you just changed it it's to the fifty five dollars there's you know",
    "because we have an option for you to the store restart your services we can either process process the whole fifty three dollars and seventeen been cents are we may process a payment arrangement spent which will you know just to just require an initial payment and then you pay the pay the rest on um on on the one the six six",
    "i understand your stand alright so let me give you the instructions on on how to complete the unlock the unlock you know luggage of the device it's very",
    "aah may i know what number are we going to remove it removing this account",
    "aah if you don't mind me asking aah did you may i know the reason why you want to remove the move the numbers on your account",
    "yeah yeah i will pay it as well it as well if i have it yeah my sister is why that why that as well ma'am i can see let me know let me know if you're you were able like to change the password",
    "in the and that way that you can get the the service of a mobile hot spot husband's him between today until and the following payments i know and september the fifteen just for for the line thirty three cents three cents",
    "hmm hmm so you need to update for sure so fair fair so it's asking you to update the system",
    "okay i got to be more than glad to assist you and i'm doing fine thank you for asking this a referral raphaela and how your self",
    "you can try going to a store and or and uh asking for the that information okay i'm okay but you need to take all your um i um i your i d and the account account information and check if they have access to any to an information on their system because as a customer customer service we do not have um the information permission in our system",
    "and i'm doing great thank you for asking how about you",
    "alright hi sir thank you so much thank you so much for patiently waiting um yeah i was able to check here check here aah i really would get an get an error it shows device not um if you don't mind if you don't mind me asking 'cause this phone is uh i'd be way older phone meaning it's it's a non ticket phone ticket phone okay now aah before you for it was activated to cricket where it was activated to read it before",
    "okay uh well yes oh just to confirm novia on the same thing that that one is happening at the moment is that you are that you are trying to access in your cricket account account and uh it's a it's asking for a security question anything anything that is why is that correct",
    "uh yes maybe that ladies yeah what nice to me the thank you for thank you for that information let me go ahead and head and access here and the account account meter click okay okay so now i'm and please ya that now but i'm here in your your account in order for you to access the so because these um i'll be asking for a for a security question let me go ahead me go ahead and send to you a link in the in that link you're gonna be able to update update the information about the the security question and also amount has been been um and then we can go ahead go ahead and try it again for you to access in your senior cricket account so so i'm gonna go ahead and send this link so thanks for you but i cannot dated i mean that i mean that way you're gonna make sure it was a security to request then you have so i'm gonna send to you this link and i send it can you please can you please tell me if you receive it",
    "i'm good thanks for asking so how can i help you today miss is julia",
    "aah continue adding adding uh the additional mobile hot spot on your phone your phone as i'll be asking your patience and aah please give me at least maximum of a minute or two if there too when it gets back everything should be updated dated okay",
    "instead of being while the fifteenth fifteen dollars the system is asking for seven dollars and and fifty cents and starting next next month or your next bill cycle you're gonna pay the full pay the full fifty five",
    "perfect sir so have a good day sir thank you so much for calling trigger",
    "that's good to hear by the way can by the way can i have your phone number that you're calling about about please",
    "aah if you want let me see if we could frequent because supposedly there should be a bridge the see which page extension tension where you can just process at least an initial initial payment for example fifteen dollar initial michelle payments and then your services will be be good and l the twenty six we have that but but i need to further check your account if if you'd be able to um process any payment would you like to do bridge bridge bridgeport or would you like to really process the whole the whole amount",
    "good evening welcome to cricket my name is name is danielle how may i have the pleasure to speak to",
    "good evening",
    "good good",
    "good evening good evening you mean welcome to cricket my name is danielle how may i have i have the pleasure to speak to",
    "good and good evening log on to",
    "all right so i'm glad i was able to able to assist you regarding with this concern kennett kenneth and just found a reminder that the account account is still good to go up until on the twenty third of august and",
    "i'm sorry about that but aah aah for this time aah and the removing of the numbers were going to have this have this removed for good",
    "yes yes okay just give me one second while i'm doing that okay okay i know this is not their whole jack and and rather leaving uh ticket against you so this coming fifteen of september temporary just going to have to pay twenty five dollars okay okay uh since your auto pay or or the credit card in your auto pay was already updated updated you don't have to reno because because on the fourteenth of our august i mean in september it will to process an autopay which is twenty five dollar five dollars only okay",
    "um your your you do the only way to get another phone would be doing an upgrade an upgrade",
    "okay okay okay okay we're going to be doing had trouble troubleshooting your phone phone from my end aah",
    "okay so what we're going to be doing is that",
    "we're gonna go ahead and work with your phone assist may first day i i was just give okay verizon had a different phone number because uh we need we need to to turn off the phone for two minutes two minutes so what we're going to be doing be doing we're gonna turn off your uh we're gonna need connect the call with you offer to be able to turn off your phone for two minutes then you're going",
    "hi how are you doing are you doing today glenda",
    "your you're most welcome and thank you as well family ma'am i i went to i would appreciate you aah you know doing the thing on the website side aah",
    "doing",
    "thank you okay so at this moment this one no we're already reset the password and i'm good and i'm gonna send you this as well so i sent you an send you an add the you should be receiving some text messages messages telling you right let me check one one is going to tell you like to change your pin inger's your security question that's in case that you need to still still change your password and it says that has been already reset reset so let me check",
    "be assigned as then as aah as far as a text message okay",
    "hmm okay i have noticed too is to that yes you made a eighty dollar payment payments on the twenty eighth of july along uh let me just check so far what happen happened on that payment because right right now the account let's do says um you'll see of fifty three dollars and seventeen cents that and so let me check about that months then okay and okay",
    "and how is your day going so far bye",
    "or due date due date okay alrighty and your services everything everything working fine so far",
    "okay okay all right all right make sure to do so far as you go to the start store just type in the website and a request request directly from the um the corporate i'm trying quarters okay",
    "so far so good good how are you",
    "perfect so far so for that one richard um i was happy in a p m m happy right now and i'm able to uh i owe uh i i was able to aah help you with your when you reach paid now and for that one for that one thank you so much for being part of pickup farmin family retried have a great day and stay safe then bye",
    "excellent to adam and my girlfriend go for today was to help with this whole older h as in just concerns did i accomplish that so far",
    "right okay but okay but the the internet's working so far right services or sir working",
    "thank you how's your day going so far going so far today miss shannon",
    "all right so john john high thing for settled here for today anything anything else i can further assist you with",
    "yeah usually we do have um you know standard high third highway time but then it's really good to know that know that i was um able to receive your call and i would oh and i was able to help you see what so again again i assure you that everything is good now you don't oh you don't need to worry about um your device anymore it's completely unlocked and you can use it to um um different device anytime you want okay",
    "a high welcome to cricket like it's frank speaking how can i help you",
    "within the billing cycle so that you have like you have like eleven days to use this fifteen gigabytes or whatever whatever if ever in case you aah aah aah you did that used all of the fifteen gigabytes but aah any remaining data for the for the hot spot during the time it will not move over on the next the next bill okay guess every every every cycle review refresh so high so how about this aah do you have access to you just to your app right now if we could app or or",
    "it's an a station high anastasia patient how is your day so far",
    "a high aah hi aah welcome to cricket frank speed frank speaking how can i help you",
    "remember to think the customer further business a high hi welcome to cricket nation this and this is lovely speaking how can i help how can i help you",
    "already used the three gigabytes right so in this case there's two plan uh three funds that you cannot grito so there's so there's the forty dollar plan the forty dollar print provide provide you at ten gigabytes a bite of high",
    "okay and now i will say oh so i will provide you the option that we so that we have to choose a security question i will need you to select this is like one of the member i made the answer okay so the options are where did you go the first the first time you fully on a plane who is your i have your first bus in what city do you need you majors post or significant other and what was what was the first name of your best friend in high school",
    "it is also not the right for the four digit pin code um if you want we can answer answer here a security question it says here here what was the first name of your best friend and i in high school",
    "okay well let's pull up your pull up your account first together aah are you calling from calling from el a phone number ending seven eight eight eight eight zero",
    "laurie alright thank you aah if you can go back to your my cricket app app",
    "aah we're trying to make sense make some changes",
    "see here i can see your payment that you made aah aah just recently for forty eight dollars there's",
    "aah yes ma'am that covers actually or two or two lines on the account",
    "that way comparing to aah getting the other one ma'am aah aah left behind but anyway if",
    "aah",
    "okay okay so we have fifty five dollar dollar unlimited plan or we also have to have the sixty dollar unlimited the plan aah",
    "okay and that is for which aah uh which rate plan aah which phone number number the four one nine six",
    "okay aah currently currently you both have the thirty dollar hello plan if he switching of fraud aah from thirty to sixty your your monthly will be ninety dollars",
    "all right you're welcome ma'am please thank you so much for calling have a great day stay safe",
    "thank you so much and i'm glad i was and i was able to resolve your concern by uh now so that you can now enjoy the retail services once once again nicole this is berna and thank you purchasing cricket and have a great day bye",
    "cricket wireless is just like sandra have a great day take care",
    "simply by unlocking or can your device and once again johnny thank you so thank you so much for being patient you have a great day but if okay but if you stay safe",
    "okay great great sir thanks for being part of the cricket family remember remember my name is edward was a pleasure to assist you this year and hope you get better of your allergies i just",
    "have a great day",
    "elsa how a wonderful rest of the day the date and great weekend okay",
    "you're welcome i'm glad we're able to resolve it twelve your concern for today kathleen thank you so much how much again for choosing cricket this is arlene and do you have a great day",
    "just a great week quick recap um corban you calling because one because you wanted",
    "great day",
    "um i can pull i can pull up i can pull up the account with the serial number",
    "um april was able to able to pull up an account april oscar check upon checking here this device was activated dated on november seventh seven and has been deactivated on of them november nine and the last the lines of service is uh of this device why is this just three days so that",
    "um um it should be i m e i",
    "um",
    "um hmm",
    "okay and i can help you with that disconnecting this line aah and uh robert okay uh since you you uh paid for this line this line okay either you can uh they uh they can you can use the paid service service okay on this line i can i can set this aah cancellation on on aah the next cycle you also have also have an option you can remove it um it may immediately but there will be no refund aah no credits but you want charge one charge for this line anymore and your bill will be they will gonna lower down as well",
    "eighty dollars hours correct and then uh since we are since we are prepaid to after you made a payment as july next my next payment is supposed to be august and i understand you mentioned you try to me to me to make a payment but those payments you attempted to come to the process yesterday and even on the fifteenth fifteen or a t and those payments were made we're failed payments your bank is declined declining the payment that's why it did not go through so and so this august they'll um um the line become suspended the cost of that but then i checked also a check back the not the eighty dollar payment that you have before you actually actually are in a sixty dollar plan so uh let me just check if check if you have uh any add on here here okay one moment",
    "yeah i uh from what i can see here the payment you you made for eighty dollars last month right right that's because of the hot spot uh you uh you were in a sixty dollar plan before then then you also have a mobile hot spot add on on so um six and sixty dollars plus ten dollars that sounds comes up to seventy dollars right right and then um there there was a reactivation fee when you made a payment the payment on july so that's so that's",
    "supposed to be yes from the from the eighty dollars that you pay for it includes the the reactive agency because your due date three eight three is on the eighteenth before four and then the payment was made on the twenty it does make makes your do every month to the nineteenth um",
    "correct and and that's why your bill also lowers down to down to um fifty three dollars and and seventeen cents once",
    "allowed to get our customers summers card number over the phone that's why we're no longer in you're allowed to separate the numbers because of the payment payment so it depends on the billing cycle michael aah aah for this one for your account account today is the last day of the billing side billing cycle from last month tomorrow",
    "alright let me so happy i was able to work with your account in your account today and help you out with your options again this is hazel thank you for your time calling cricket cricket and have a nice day okay",
    "hello hello april how can i help you today",
    "all right okay anything else anything else for today april before i let you go",
    "welcome to create condition mister here how can i help you today",
    "perfect sir so any notes here to help today",
    "we process this this uh fifty three dollars and seventeen cents moved and moving forward your payment will just be just the fifty five dollars because you were in a we're in a fifty five dollar plan today no longer",
    "hello thank you for calling cregan wireless is so since all extenders speaking how can i assist you today",
    "tell me how can i ask you can i assist you today sir",
    "thank you so much johnny how can i help you can i help you today",
    "on the fine has actually my um start today today my first",
    "all right that's good good so kathleen what we have done today today mysteries accessed fully reset your password for your to the account so you have access to it now and and just a reminder your next due date september september seventeen bill is forty dollars and dollars and you have autopay set up which is very convenient for you it for you is there anything else i can help you you",
    "let's see right but just to make sure can you um can you can you tell me what phone is it today today or is it still",
    "alright alright there we go yep the account is already set here in a brief a bridge peso again at tomorrow right on or before know before the twenty fourth aah forty five dollars and eighty eighty one cents will be the uh ticket amount the letter so you make the payment through the app the six and one or one or the forget wise it talking up so they could pay or you or using your apps it's all set right now okay that'd be was able to set this up here for you today today would there be anything else let me know here",
    "i'm trying to get this get this is valerie who do i have the pleasure of speaking today today",
    "is a pleasure okay okay there's nothing else that i can do for you for today today",
    "all right had or using how may i help you today today",
    "hello joshua how may i help you today today",
    "by the way how may i have the pleasure to be speaking with today today",
    "aah yes you can purchase and phone right phone right now i know website but but the aah upgrade eligibility they will not aah the until till of tober fourteen so if you do it today today you will be charged full month's aah i mean for i mean full payment on the phone price",
    "yes",
    "yes ma'am",
    "the",
    "remember just hang the customer further business yeah my internet's not working on my side my phone in my phones are working size is it it it turned back on or whatever's going on there's one on it",
    "you too",
    "remember to think the customer for their business hmm yeah can you switch me over to billing billing somebody in billing",
    "remember just sent the customer for their business business messinger mary",
    "is",
    "four six four six four six",
    "you",
    "have two",
    "that option",
    "one",
    "hello yeah",
    "press perfect create a new one the one that you can easily remember",
    "that will be that would be under settings",
    "but nobody nobody told me that i have to use cricket in the six months time period",
    "right",
    "the",
    "no",
    "trying to get through on my phone",
    "okay okay i'll be different definitely get in contact with the word the department with this so",
    "hmm",
    "yes ma'am because you because you are on automatic payment",
    "yes ma'am",
    "sure",
    "okay so you said you said you forgot the well he forgot forgot",
    "well",
    "is",
    "remember just sent the customer for their business business oh hmm",
    "and all i did",
    "i'm sorry",
    "the billing cycle go ends oh yeah",
    "everything",
    "me",
    "your text oh go ahead go ahead",
    "repeat that",
    "ma'am no hmm no it's not your mistake it's the phone itself south sometimes",
    "yeah i lose that yeah l a",
    "thank you can you please provide please provide me the card number",
    "yes",
    "but",
    "can i have the four digit pin please",
    "good morning how are you doing",
    "i'm sorry",
    "hmm",
    "hmm",
    "it for",
    "yes",
    "correct fifty dollars enter his this is in charge yes",
    "but",
    "they go through general",
    "what's",
    "hmm",
    "hmm okay so the paper your device is completely you mean ma'am so that you can block them",
    "me",
    "you want me to go on the payment",
    "forty four",
    "yes sir",
    "assistance",
    "yes ma'am",
    "oh a lake charles out valid shop all six oh one four",
    "you very",
    "the number that order already exist",
    "let me see",
    "aah i can i can i i do this every month",
    "how much is the total amount that you pay all your bills bills michael",
    "please be there please bear with me okay verifying the exact unlock code you have would you have so as part of the policy of the unlocked lock code we can",
    "more",
    "yes i am",
    "glad",
    "child",
    "okay and your four digit pin code",
    "oh who when be hoover when unlock it",
    "i i rebooted it wasn't was an x submit",
    "mobile",
    "hmm so since you have should have the wifi in your wifi is working at your home your home so try to connect your phone so that you can that you can update the system of your device",
    "remember just hang the customer for their business um seat or for denny",
    "me",
    "two two this thank you very much for",
    "those",
    "dollars credit in your account account",
    "i",
    "hmm not",
    "oh",
    "the",
    "all right okay all right thank you how about the i c c i d or the i d or the sim card did you change it as well",
    "let me see hold on in ten dollars on the old one",
    "you",
    "oh no",
    "phone are you looking to upgrade to",
    "yeah",
    "for",
    "new sixty sixty dollar plan",
    "well i see the account and i don't and i don't see any card on file to be honest with you mister mister anderson i see that you don't have autopay",
    "you hear",
    "still once",
    "yep i yep i restarted",
    "yeah already in your account so let me go ahead let me go ahead and check the balance that you have right now",
    "i i will walk you through ovaries very so stay on the line let me send you the link first first",
    "nine",
    "no",
    "yeah right",
    "one from that from that number and it still wouldn't let me call you call you guys either",
    "hmm",
    "and also and also does include",
    "okay it's just four digits so what you what would you want me to try",
    "so",
    "after",
    "and i have been with correct cricket oh well fifteen years",
    "hmm hmm",
    "no",
    "your accounts together gather",
    "so",
    "got it thank you you may i please a minute or two just want to want to make sure that you are connected to the struggle trying to start working",
    "yeah",
    "hold",
    "yeah just miss bernadette's i check the",
    "okay let me go ahead and start processing the seeing the payments it's currently loading",
    "um as of now i now i had two but i just had three earlier there",
    "told",
    "okay just stay on the line i am just i'm just working on the account can i put you on a you on a brief hold allowance minutes",
    "hmm",
    "nine",
    "alright there you go",
    "thank you i got this one here and this is for me to for me to attempt to get the account can i have the four digit did it in please",
    "the",
    "hmm hmm",
    "for",
    "hmm",
    "anything",
    "so you're saying so you're saying the motorola that i'm trying to use is not bad not compatible with",
    "well and then i'll have to go there go make those somebody virus they phone so i can make some to make some phone calls calls aah and which is which is ridiculous you know they did they didn't go they were gonna turn stop they could've new sent me a notice no it was saying that they were gonna turn the phone off that's only courtesy they sent me a puppy i thought they sent me a notice last month the month a month and phone and my errors has not been handle want every month they sent me a phone your phone bill is bill is due on your bill is someone telephones so so and then this time you get up and it don't who is all",
    "christine part first",
    "and what was the zip code you put in",
    "my business",
    "to reset my phone oh",
    "correct",
    "tell me that i have a text and it doesn't make any make a note",
    "as in the reservation sure",
    "no",
    "the",
    "i",
    "what's the card number",
    "i said",
    "yeah it's forty one forty one hundred is my husband",
    "to",
    "yeah well yeah kind dot com sir like i have a link the internet on my phone phone but i didn't know if you go if you guys like so like a tablet with but with the phone line on it that would be that that would have unlimited internet or if you guys if you guys so like like a little it'll aah hot spot box type deal do you",
    "it",
    "hundred",
    "under the settings on there about",
    "so",
    "okay can you check if you have received receive any text message",
    "hmm and that's the i m e i for i for the replaced or the",
    "hmm",
    "from this towers so all we just need to do is to is to undergo with settings to check for travel for troubleshooting okay",
    "can you please roger to dial to a different number",
    "okay first lie i do a i do",
    "let me give you some okay so give me give me what it will be do you have do you have any",
    "oh so",
    "yeah the payments so and so any posted in different request stephanie very bad service",
    "oh",
    "it don't matter no",
    "all righty miss came up let's try the phone the phone on okay",
    "any color i won't correct",
    "so",
    "hmm",
    "the process",
    "is there",
    "okay perfect perfect yep so that whatever everything code you enter there is the note ten code and go d on the account so",
    "hmm",
    "yeah",
    "got",
    "hmm",
    "hmm",
    "mm hmm",
    "so when so when you use up that percentage of the the increment of the fifteen",
    "i i just need your four digit pin please then please",
    "they got the they got the wrong card number",
    "seven",
    "make a phone call",
    "what i'm saying that they did give you a mother give me aah oh like that is why they on it unlocked we don't like the phone aah",
    "is",
    "wait",
    "hmm yes i do",
    "hmm",
    "yeah it did one",
    "is to put your phone your phone number and the amount the card information metion and there will be no charges for that for that",
    "hmm sorry sorry",
    "ma'am yeah",
    "yes ma'am",
    "m e i what you said my name",
    "you",
    "i'm sorry",
    "the phone number",
    "so so basic basically and just to confirm may sir may i please have the i m e i of the may i have the phone",
    "connection"
])


actual = [
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS"
]

f1 = f1_score(actual, preds, average='macro')

acc = accuracy_score(actual, preds)

print([runtime, f1, acc])

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/30 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1200
  Num epochs = 3
  Total optimization steps = 300
  Total train batch size = 12
running astray


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

[28.515502946999987, 0.630870871602812, 0.631184407796102]


In [4]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score

import pandas as pd

model = SetFitModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_test_shot.xlsx")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)


trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=12,
    column_mapping={'Class':'label', 'Text':'text'},
    num_iterations=50, # Number of text pairs to generate for contrastive learning
    num_epochs=5 # Number of epochs to use for contrastive learning
)
start = time.process_time()
trainer.train()
runtime = time.process_time() - start

preds = trainer.model.predict([
    "hello hi welcome to cricket mission minimum my name is jimmy how may i have your name please",
    "that's still i'm still a was able to clear it to you uh thank you for thank you for calling",
    "welcome to the ticket to take your permission my name is add may i have your name please please",
    "all right well thank you very much for calling through good i'm good good i'm glad i was able to add a hot spot and of and uh basically answer the question about the bill so this is so this is now once again thank you very much for being the best part of cricket you do have a great day bye for now",
    "welcome to cricket click admission my name is john can i have your name please name please",
    "hi you're welcome hi welcome to cricket things on this is josie may i have your i have your name please",
    "okay i'm very happy with i was able to resolve your concern by canceling one last two line two lines on the account this is laurel again and thank you for choosing crickets tuner you have a wonderful day and stay safe",
    "hi thank you for choosing tricking tricky my name is daniel with them speaking today",
    "welcome to cricket assist this is john may i have your name please",
    "hello welcome to cricket nation my name is my name is john how can i help you today",
    "hello welcome to cricket my name is jay jade may i have your name please hi there welcome",
    "hello welcome to cricket ma'am how can i help you can i help you",
    "hi welcome to click in my name is jay may i have your may i have your name please",
    "thank you so much i'm glad to i'm glad i was able to resolve your concern and by the way by the way i my name is an aerial when that thank you so thank you so much for calling i wish you",
    "okay aah so so yeah i guess it once the patiently for every time i'm happy happy with able to assist you fifty and so i'm sorry so thank you so much for calling cricket have it for you have a great day",
    "hello welcome to welcome to cricket my name is anna may i please have your name",
    "welcome to create an agent major in my name is daniel with whom i have the pleasure to speak with take this",
    "welcome to cricket cricketing tion my name is rafi l and my hair in i have your name please",
    "thank you for calling verizon wireless my name yes my name is better which one will i be having a pleasure let's you're speaking with",
    "hello hello welcome to cricket this is hazel how can i help you you",
    "you're welcome to cricket to cricket this has been nonna may i have your name please",
    "all right you're welcome ma'am please thank you so much for calling have a great day stay safe",
    "hi welcome to tickets nation my name is my name is berna how can i help you today",
    "welcome to create condition mister here how can i help you today",
    "yes sir well actually well actually the payments is actually prester right now now what i recommend sir is to turn off turn off the phone and turn it back on and everything should be working you're welcome sir",
    "welcome welcome to coordination this is jonah can i have your name have your name",
    "welcome to cricketing coordination this is josie may i have your name please",
    "good evening welcome to cricket my name is name is danielle how may i have the pleasure to speak to",
    "good evening good evening you mean welcome to cricket my name is danielle how may i have i have the pleasure to speak to",
    "hmm hi robert how you doing today",
    "also high hi how can i help you",
    "hi there very welcome to the technician is the one let me have your name your name please",
    "hi gary how can i help you today help you today",
    "if you upgrade as per as per checking here you're eligible for a device that hi discount so that means if you plan you plan on upgrading your device you can definitely and i only have a discount as long as you're logged log in your account over cricket wireless dot com com you can choose your device will recover device device if you should be able to receive a discount discount",
    "hi good morning this is this is chris from cricket how can i help you today okay",
    "hi does how does right how can i help you",
    "hi welcome to coordination this is christian christian can i have your name please",
    "hi hi welcome to the technician this is using sure we can have your first name please",
    "alright hi sir thank you so much thank you so much for patiently waiting um yeah i was able to check here check here aah i really would get an get an error it shows device not um if you don't mind if you don't mind me asking 'cause this phone is uh i'd be way older phone meaning it's it's a non ticket phone ticket phone okay now aah before you for it was activated to cricket where it was activated to read it before",
    "okay well let's pull up your pull up your account first together aah are you calling from calling from el a phone number ending seven eight eight eight eight zero",
    "alright let me so happy i was able to work with your account in your account today and help you out with your options again this is hazel thank you for your time calling cricket cricket and have a nice day okay",
    "thank you very much you're very much for calling to coordination that i made a speaking we can you please primarly for your name",
    "perfect sir so thank you so thank you so much for calling cricket my",
    "perfect sir so have a good day sir thank you so much for calling trigger",
    "that's good to hear by the way can by the way can i have your phone number that you're calling about about please",
    "hello thank you for calling cregan wireless is so since all extenders speaking how can i assist you today",
    "yeah okay arroba don't worry don't worry i got to be more than glad to assist you and let me let me take a look here and take care about your information information may i have the phone number associated with your with your account please",
    "okay well my so well my goal today was to assist you added the the uh rate plan for a much better better one and that way that you can get the the mobile hot spot in your two services like this line then i go was this so still help you out today and referral and thank you for choosing which is in cricket by a services provider that number remember my",
    "hello hello april how can i help you today",
    "hmm sorry to hear that your internet as well this off no worries i can help you with that so so before we proceed are we working on the number and number and in two nine six seven",
    "or you have the wifi on your help on how on your house",
    "miss green that us how can we help you",
    "oh okay i do i do apologize to hear then and don't worry i can help you may i help you with that let's go ahead and take a look take a look can i have your phone number mother",
    "thank you thank you very much for that information okay okay let me go ahead and your help you help you try to unlock the device here and just hold on hold on just let me open the total tried online online that device as well yeah hold on",
    "okay sure and can i help you out can i help you out in the hospital i can give me your name",
    "perfect sir so any notes here to help today",
    "awesome thank you and how can i help you by the way by the way",
    "hmm sure megan and i can help you with that disconnect one of your line can i have the have the number that you would like to uh removing remover",
    "hi welcome to click to cricket my name is jay may i have your name please",
    "hello hi hi hi this is this is jay from cricket service actually on the phone phone with rochelle",
    "all righty well thank you very much for being for being the best part of cricketing in my name is jay they do you have a great one okay stay safe",
    "welcome welcome to cricket my name is jay may i have your name have your name please",
    "name is jay and have a great day goodbye for now",
    "hi welcome to click in my name is jay may i have your may i have your name please",
    "alright well thank you very much much for being the best part of cricket again my name is jay have a great one today and stay safe okay",
    "okay this is jay getting have your name your name",
    "hi welcome to to cricket mention this is jay can i have your name",
    "you're welcome and thank you so much for much for calling cricket this is jay you do have a wonderful day",
    "okay aah currently currently you both have the thirty dollar hello plan if he switching of fraud aah from thirty to sixty your your monthly will be ninety dollars",
    "hello",
    "hello hello",
    "thank you um have um hello there can it thank you so",
    "aah hello thank you so much for so much for patiently waiting um let me just",
    "hello detail yes",
    "hello hello thing as i'm not sure you're calling freakin' wirless this is joe it says y'all speaking how can i help you today",
    "hello sir thank you for patiently for patiently waiting on the line does a check check on the account here and and uh i'm checking here there isn't as actually a payment that was made on the thirteenth fifteenth but what i can see here is that there was a there was actually one attempt of thing of a payment payment here last twelve of of uh august and um you hmm you made the payment online i'm i correct",
    "hello welcome hello welcome to cricket my name is jen how can i help you",
    "hello welcome to welcome to try to wireless my name is jennifer may i have your name please and please",
    "laurie alright thank you aah if you can go back to your my cricket app app",
    "aah aah okay normally would cricket if you'd if you're going to be making an upgrade of of the other line we always recommend that you do the and do that for both lines so that we can take advantage take advantage of a bundle discount it it's cheaper",
    "aah two options that they can they can separate a number lorraine it's either either you can aah go to the cricket store get store the store can actually do that within the day day or we can also file file a ticket to back office back office signal but the one to call you and set and separate the numbers over the phone but but the ticket usually will take three to five business days space for a call back",
    "aah because you only have i only have paid for partial of the account account remember you have activated the and the second line just july twenty nine nine supposedly your bill is sixty sixty dollars or sixty eight i would say because there's 'cause there's cricket insurance on the other number but you only paid forty eight it's because it's because you only paying for the proration tion of those days left on the billing side billing cycle so supposedly simply aah you you paid up for for the service of the phone number ending ending four one nine six from july july twenty nine it's activation up until aah august six aah like aah august seventeen when we start and restart your new billing cycle",
    "means hmm we cannot unless unlock this device because it is only been active activated for three days with a is a cricket sam",
    "thank you so much and i'm glad i was and i was able to resolve your concern by uh now so that you can now enjoy the retail services once once again nicole this is berna and thank you purchasing cricket and have a great day bye",
    "okay and if and just gonna go ahead and check something because it seems that this phone this phone is not from cricket",
    "yeah yeah the device it doesn't displace any and he called because it's not frank frank cricket but the other thing that you need in order for you to move out to okay so another deploying other provide the provider it's your account number and the but in the password or pin mother",
    "by by the way for me to helping you out from cricket",
    "hmm cricket is a prepaid it's a prepaid service uh robert there's no refund refund we don't uh credits either the customer aah if if the if they pay the service to get a use up until until uh the next few days because again once and once we",
    "okay and that is for which aah uh which rate plan aah which phone number number the four one nine six",
    "will be the start of your new billing cycle that one oh that would start of course august seventeen up up until september sixteen so for so for example if indeed then at the middle of the billing cycle in cycle you'd wanna separate the number number they will going to prorate those days was days aah left on your billing cycle cycle as a refund and then the other line the airline uh that will be separated will be created a brand new aah bill aah billing cycle it's own billing",
    "of you will get that twenty five dollars hours with uh refer offend aah credit but if it's added on your account your account it's not a uh before for it verifying",
    "purchase by just having the i m e a april i need to pull up an account account for uh as well your",
    "all right all right still looking in here uh april just hold on hold on",
    "um april was able to able to pull up an account april oscar check upon checking here this device was activated dated on november seventh seven and has been deactivated on of them november nine and the last the lines of service is uh of this device why is this just three days so that",
    "hmm just give me a second here i will and i will just check hmm okay uh uh i'm checking here you know that our on your area yeah yeah but all the tower is working fine and active active so let us go ahead and put and then proceed with the troubleshooting of your device so did you try any trouble and the trouble shooting on your device before",
    "yeah that's why uh is it okay if you can check can check if your cellular data is turned stand on",
    "uh on the settings thanks and then you have to select cellular",
    "uh you should turn on that one",
    "hi hi welcome to cricket my name is judy how how can i help",
    "hi welcome tickler destination that is area how so how can i help you today",
    "hi welcome to welcome to cricket nation my name is kingston may ask may i ask for your name please",
    "hi welcome to cricket you should this is allen speaking may i know your name okay",
    "hi welcome to cricket nation my name is linda and then may i have your name",
    "hi welcome to click on save uh thank you so much for patiently waiting on the line dan i do apologize apologize columb wait all right all right so i just refresh the cricket account here here and i've just made made sure that the phone number ending ending in five one one four is is associated with the iphone x r are let me double check every so it's reloading one more one more time so we can make sure it definitely make sure that make sure that this will not be suspended again okay",
    "hi welcome to recognition sir my name is raffia may i have your name please can you hear me hear me okay",
    "hi welcome to cricket my name is name is one c may i have your name please",
    "hi welcome to cricket mission patient is john how can i help",
    "hi welcome to click to cricket my name is ryan may i have your name please",
    "alright john all right johnny i'm just waiting for a response to stay with me okay with me okay we're",
    "thank you john y",
    "um if you have a new device you have a do you have a device john because of course with she for us to activate it line we need to have a device",
    "well currently currently john we have uh you can get zero dollars zero dollar devices if you transfer from from another service provider to cricket",
    "welcome to cricket nation my name is john is john can i have your first and last name",
    "you're most welcome ma'am is your your services once again this has been john ma'am thank you for being the best part of cricket nation keep safe okay",
    "oh hi must john how may i help you today day",
    "cricket cricket mission my name is may john and your name please name please",
    "you don't use all that all the gigs that's make an example john use my use five gigs can will be missing any and it will be lost because it will not be like stuck they will be lost you lost",
    "hello good morning my new welcome to cricket nation my name is john john how can i help you today",
    "alright so brandon having that i was able to give you the breakdown of the charge the charges an option to separate the number i will note that in your account if in case you go to the store to the store they can pick up where we left off yes okay",
    "be i won't be able",
    "all right let me just check all right april for this one i this one i'm able to unlock there's an unlock code or hold are you ready do you have pen and paper with you",
    "unlocked not unlocked uni you it will be able it will be able to transfer it",
    "okay okay three five six one and just a disclaimer klamer robert if in case in the future okay okay if you would like to reactivate this line you only have only have twenty nine days you're going to pay whatever the whatever the plan price and fifteen dollars for the activation should be the one twenty nine days when able to read would you reactivate the same number but i know you can and get a new sim card and activate a new number instead instead okay",
    "okay and i'm glad i was able to resolve resolve the issue the phone number ending in three five six one six one completely disconnected or removed remove your updated balance from one from one eighty six it went down to one fifty six fifty six dollars uh with your autopay do they do on uh september eight okay and",
    "aah if you want let me see if we could frequent because supposedly there should be a bridge the see which page extension tension where you can just process at least an initial initial payment for example fifteen dollar initial michelle payments and then your services will be be good and l the twenty six we have that but but i need to further check your account if if you'd be able to um process any payment would you like to do bridge bridge bridgeport or would you like to really process the whole the whole amount",
    "able",
    "and there there you'll be able to aah click click on this and that'll be it to you uh you will have",
    "the pleasure to speak to",
    "be more than glad to glad to assist you with that um we do i have the do i have the pleasure to speak with",
    "welcome to the quick admission my name is edwin when please tell me who i'm glad to speak with today with today welcome to the combination they sure my name is edwin please tell me tell me where i'm glad to speak with today speak with today hello hello hello hello welcome to the green one that's mention this is that when that when you tell me how may i help you today",
    "amount of the night ninety dollars that you can also settle through the my through the my cricket app or through the cricket wireless dot com dot com and there you go run the congratulations gotcha lesions as we speak i have successfully successfully take care of it for you and it's now and unlimited plan i mean unlimited data for the for the fifty five dollar",
    "and because and because of the following four nineteen example i had one sent you situation i'm going to be direct with you i'm going to speak speak straight up to you i'm sorry i don't mean to be done by the one i don't want to be rude talkin' about this no no no no i'm on my",
    "thank you for calling thank you for calling quickening sure my name is is uh with one would wanna speak it right now",
    "and that's my screen broke on my so i put my sim card into an iphone iphone and it's saying that i put it is but it is not you can make a payment from i mean as being payments aah on um on friday but speak so so i'm not sure what it's saying you make a payment",
    "hello thank you so much thank you so much for calling crickets mention this is jacqueline how much do i have the pleasure to speak with",
    "hi welcome to cricket nation despaired speak speaking can i please have your name hi welcome to cricket mission my name is bert can i have your name please hello hello",
    "hi hi welcome to cricket nation this this is shane speaking speaking may i have your name please hello welcome to create because my son is this is shane speaking speaking may i have your name hello hello hello welcome to cricket mention this is shane speak speaking may i have your name hello hello aah aah memory sir if you could hear me i just wanted to it's inform you that i'm not hearing any response once from you hello hello hello ma'am are you sir if you could hear me again again i just want to inform you that i'm not hearing harry any response from you hello hello hello welcome to cricket nations this is james shane speaking may i have your name okay hello aah memory sir if we could she can you hear me again i'm not hearing and a response once you this this will be the devices might disconnect the call now to accommodate more customer customers reading on here she has to further to further questions about your account the service finish the start star the number again okay thank you",
    "tell me how can i ask you can i assist you today sir",
    "all right so i'm glad i was able to able to assist you regarding with this concern kennett kenneth and just found a reminder that the account account is still good to go up until on the twenty third of august and",
    "all right so john john high thing for settled here for today anything anything else i can further assist you with",
    "okay great great sir thanks for being part of the cricket family remember remember my name is edward was a pleasure to assist you this year and hope you get better of your allergies i just",
    "righty don't worry i'll be more than be more than happy to assist you with that today can i have your name and name and the phone number",
    "yes ma'am is there anything else i can now i can assist you with",
    "all righty miss and do this as well recheck technical side to make a payment and we also cancel the line cancel the line ending let's see fifty sixty three thirty four it my goal to assist you with to see what this today have a compass that",
    "okay i got to be more than glad to assist you and i'm doing fine thank you for asking this a referral raphaela and how your self",
    "hello welcome to break it my name is danielle how yes how can i ask you if you have the name",
    "uh hi welcome welcome to aah cricket frank speaking how can how can i help you",
    "hello welcome to cricket my name is yes my name is jen how can i help you hello this is jen from cricket how can i help you today help you today",
    "hi hi good morning welcome to cricket patient this issue this is your main how may i help",
    "thank you so much for calling crigger wireless my name is self itself and how can i help you today",
    "hi welcome to welcome to cricket mission my name is darcy may have your name have your name please",
    "uh my name is ernest was a pleasure to assist you thank you for calling the recognition",
    "welcome to welcome to",
    "hello welcome to coordination my name is roger roger with who i have the pleasure today",
    "hi hi thank you for calling cricket my name is bell how can i have your name please",
    "welcome to cricket to cricket my name is allison can i have your name",
    "welcome to clarification my name is twelve you're welcome have your name please",
    "all right you're welcome to verizon wireless the last one of the phone how can i assist you today",
    "welcome to cricket nation",
    "hi would welcome to coordination this is jessica how can i help you may i help you today",
    "welcome to bring",
    "hi welcome to clarification",
    "so i won't take too much of your time to ya yeah again thank you so much for calling cricket today good today my name is joan us once again you do have a great day and you take care as",
    "okay don't worry again thank you for calling verizon wireless have a great day",
    "hello welcome to correct to cricket this is my nonna can i get your name please please",
    "welcome welcome to the condition my name is i said it has a pleasure to speak to speak with doctor hello",
    "me too me so um for this one this one i can see your message jam that within within seven days seven business days you'll be able to receive an email l so let me just check check when was the um other line was active activate it",
    "thank you thank you so much miranda so let me give her let me go ahead and pull up your account here first alright alright so i got no and access of your account here aah let me check your mail hot spot aah",
    "thank you so much hold on much hold on alright thank you so much make much nichole for patiently waiting apologize for the hold",
    "all right thank you okay",
    "okay thank you you too goodbye",
    "okay i'll go over there i there i appreciate your time thank you",
    "right guess she still have it i don't know what she through the sim card the sim card away once you through the phone away she probably good thank you take it out so she went through a whole new carrier and i was and i was trying to place an instead of suspending it bring it",
    "too thank you then i have a wonderful day",
    "no you helped a lot thank you very much for much for your help",
    "too thank you bye bye",
    "no that'll be fine thank you so much",
    "so that's all thank you",
    "that's it that's the reason i'm just i mean i i've done all the all these years and i had a land base price this phone you know and i only a thirty minutes thirty minutes long this is of course he can't tax or anything or anything it went up to sixty bucks a month what were you guys are half of that and so that's that's you know you made me come to you and i should have i should've did this a long time ago but thank god thanks for anyway and i appreciate it and aah aah i'm gonna go here and one comments on it aah i can tell you care it's coming over and i really appreciate appreciate that i live by like that so thank you thank you",
    "all right uh alright awesome thank you so much",
    "okay thank you okay thank you so much",
    "uh uh thank you",
    "okay no problem thank you so much",
    "thank you so much have a blessed day",
    "okay sounds great thank you right thank you",
    "but again i appreciate you thank you",
    "i see okay i appreciate it appreciate it thank you so much for your help",
    "okay great okay thank you so much i much i really appreciate your help",
    "okay all right well i appreciate it hi everything you in pre check your help and i have and i hope you have a good rest of the day",
    "i appreciate you",
    "yes sir okay okay yes sir no appreciate it",
    "thank you sir i appreciate",
    "no that's all i needed appreciate you take your",
    "that i mean they were whenever i go get go get the the new phone i'm on i'm going to bring my laptop so they can so they can do i can do that then you know the other things things but i really appreciate it",
    "yeah but aah but anyway thank you so you know to ask you to do that to tell me i really appreciate it",
    "no that's all i wanted to know and i i know and i really appreciate your help and have a wonderful rest of the agent",
    "um my cell phone and then i it's a unlock i'm not not my my and then the i go to the do the um that is well i can use i can use the country the uh the sim card sim card so i'm good thank you so much for me let me",
    "depends on the phone okay hmm hmm okay sounds good thank you very much that's all i needed to pay okay",
    "you haven't you have a good thank",
    "see okay okay well good thank you so much i i really appreciate your time with more ticket have a great day",
    "okay very good thank you",
    "all right so sounds good thank you",
    "okay sounds good sounds good thank you",
    "okay sounds good thank you",
    "all right good thank you so much so much",
    "good thank",
    "thank",
    "okay okay i think i'll write it down now okay now okay thank you so much",
    "thank you hazel to hazel you as well",
    "okay okay all right thank you",
    "no thank you",
    "to thank you for helping me you have",
    "okay oh oh gotcha okay thank you it's hundred percent percent then let me take okay then you're calling to see if there's any kind of now this hot and he got it yes yes hundred i'm waiting on again",
    "thank you you too bye see i",
    "so that's all thank you",
    "thank the customer for their business aah send me a good morning or not i can i can let her i'm applying from cricket ticket",
    "all right thank you",
    "all right thank you very much",
    "okay okay so yeah i guess those details so much for everything all right thank you so much for calling frontier have a great day ahead bye for now",
    "okay i okay all right thank you so much and i'm",
    "all right thank you so much and you as",
    "okay all right okay all right thank you",
    "yeah it says aah is aah yeah uh bridge payment but which says uh right now right near from cricket your can setup bridge right thank you all right thank you for your bridge first page by payment you you need to make your bridge permanent by eight twenty eight twenty the by the twenty four",
    "all right thank",
    "okay sounds great thank you right thank you",
    "okay alright thank you all right thank you so much",
    "remember to sync the customer for their business uh april stoner",
    "it never had a phone number phone number i have there's another phone number that number they can't do anything uh i'm going to i'm turning on now you can't look it up at the serial number",
    "okay let's see uh uh settings harry um um ready um hmm about phone here we go it does not have a phone number it says unknown it gives me a model serial serial and i m e i number",
    "uh",
    "okay i okay i don't know when it says do i have another phone phone let me see um hold on uh all right right which was turning on all right settings so i have a serial number on not on this one",
    "uh i'm ready",
    "uh huh",
    "we're just saying the cost the customer for their business hi uh donna i will call i was calling because my internet is off",
    "it's saying it's saying it hasn't i i can't connect to it but do it for the past like uh five to ten minutes now now it says it down",
    "yeah connect the internet internet you're offline check your connection and and when i press crap um tap to uh um we uh uh try to connection or cellular cellular data connections are not available during the call what's goin' on with this phone can i can i get now no internet connection and i was trying to trying to go on my email",
    "okay thank you miss thank you to send you",
    "okay thank you",
    "okay thank you so turn off the phone for two minutes and turn it back on okay thank you thank you bye bye",
    "okay okay thank",
    "okay okay thank you",
    "okay thank you thank you",
    "okay thank you so much you so much",
    "okay thank you very much i appreciate it",
    "okay thank you so much i'll be careful with the thank you",
    "okay thank",
    "sixty eight dollars okay alright and so if i split the but this like i like i got this for someone for and for the birthday do i qualified for the twenty the twenty five dollar thing",
    "okay okay okay alright cool",
    "alright",
    "alright i just wanna it's one of two numbers aah because",
    "all right alright",
    "alright thanks sounds like something",
    "alright thank you got it thank you very much",
    "alright thank",
    "thank you also have to find that",
    "the app to absolutely not you've been so helpful thank you so much",
    "to thank you for helping me you have",
    "okay then well thank you so much",
    "okay that's what i needed to know it to know all right thank you very much much",
    "uh no no thank you",
    "very good thank you so much",
    "that's all thank you sir",
    "hmm hmm thank you",
    "no i'm good thank you for yeah i appreciate it",
    "so much higher i appreciate that",
    "of great thank you so much any",
    "thank you you too sir",
    "i'll call you back thank you",
    "um like i mean i don't think you thank you for that and she doesn't even charges for just phone i'm thinking that he might not needed i need it",
    "okay okay thank you so much for your help much for your help",
    "thank you very much much",
    "all right so sounds good thank you",
    "okay oh yeah so i got this this month my favorite alright okay came out",
    "okay one second and",
    "okay okay now supposed to one of us reflect needs more data than the other house how do i do that",
    "okay",
    "okay he he he wants to sixty dollars unlimited plan but",
    "six okay and then and then aah that'll be nice be ninety plus thirty and and if we if we both switched to the unlimited what what it what would that be",
    "okay so that's no deal for me i don't need i don't need unlimited okay now supposed like was like i i i got this line for real relative for the birthday",
    "okay okay and so what is it is there any cost for separating the ticket the lines",
    "okay so supposed like i said pirated read it today what would it cost",
    "okay so what do you mean less than thirty thirty",
    "okay so i don't",
    "so my next so if i so if i do not split it what's the next the next day the next bill what is going to be my next bill",
    "a great day",
    "great day",
    "a day before",
    "okay sir okay so for celisa something has to get a record of they told me to go comma falls companies get the phone the phone records or whatever i don't know how to i just need a i don't i can't see how to do that uh with uh i can't i can't i don't know how to look on your from what they said they don't is i'll go back that far it's no way i can go to somebody y'all can can print out the twenty fifth of the call them cause i made on that day",
    "i'm calling this is my third phone calls tell my aah it's twenty for some reason i'm not getting getting my uh internet can i know that that's then that's what this is the third call and this is too i two hours with the situation i don't think it's under just understand i'm kind of short on the page that i was told was gonna get paid and then it's internet is we're gonna call me back last thing would day was said make sure that all my oh my wifi aah you know i didn't know if then open rest of the distending and i still get very low bar far",
    "made my day they",
    "nope no i don't have no more i just i know you got going on how are you awesome mericans things around you're very much and you have a great day",
    "it doesn't help none and my best bet to you is this day this day and time brother you can be that healthy oh this or the port to healthiest my is my dad and the port my lives is this where i looked at it when you're time ago your time your time ago you know what i'm saying is",
    "or the day too to the whole",
    "nine alright and twenty three",
    "all right sure i can go ahead ahead and give you uh the information can so that will be able to know what you're going to do is to do if in case that you would like to suspend the line and filling up before i go ahead and do that let me ask you may i ask you how are you doing today",
    "hi how are you today ma'am hmm",
    "i am yeah i'm good ma'am how about you",
    "the door hi do you do you know how are you doing today",
    "and aah i would like to a steven for then for how long have you been using this device is active actively to cricket network",
    "thank you how's your day going so far going so far today miss shannon",
    "thank you and how and how are you doing today",
    "so your screen how long have you been using so new phone",
    "and i'm doing great thank you for asking how about you",
    "and how is your day going so far bye",
    "it's an a station high anastasia patient how is your day so far",
    "hi david good evening how are you today okay",
    "all right and how and how are you doing today with ya",
    "okay i thought i lost you aah i i hope you don't mind me asking be asking for and how are you doing today",
    "hmm how about you to try and send a text message and say",
    "lamont hyla month how are you doing today",
    "how are you are you",
    "uh t m c a nice submission t m how are you today ma'am today ma'am",
    "brian high ryan how are you doing how are you doing today",
    "very high area how are you doing how are you doing today",
    "way safe escalate would be the website and in case in case you go in and make a payment in the future you're doing you already know what to do okay",
    "hmm all right so what it as a what we will be the will be doing on the phone number ending on for one four one six nine four one nine six",
    "so yes i did yes i did how are you doing today",
    "thank you miss laura how are you doing today nice to have nice to have you on the line",
    "thank you ma'am thank you very good and how are you doing today",
    "all right and how and how are you doing today with ya",
    "hello how are you doing today mary anna anna",
    "it is if these other how are you doing today sir",
    "how are you doing today doing today salvatore's",
    "during high doreen how are you doing today",
    "very high area how are you doing how are you doing today",
    "how you doing today patricia",
    "aah aah okay normally would cricket if you'd if you're going to be making an upgrade of of the other line we always recommend that you do the and do that for both lines so that we can take advantage take advantage of a bundle discount it it's cheaper",
    "will be the start of your new billing cycle that one oh that would start of course august seventeen up up until september sixteen so for so for example if indeed then at the middle of the billing cycle in cycle you'd wanna separate the number number they will going to prorate those days was days aah left on your billing cycle cycle as a refund and then the other line the airline uh that will be separated will be created a brand new aah bill aah billing cycle it's own billing",
    "it's gonna be going to be six to eight dollars due on september number sixteen that's gonna be the regular bill yeah",
    "so are we going to be making the upgrade or not yet not yet",
    "well let me take a look at this one here here then give me just a moment let me check on check what's going on here aah the phone number you were referring to into is other one year on you're on right now ending in one two four five is that right is that right",
    "okay so aah now now i'll process first first the eight one six eight three two three five two seven hold on hold on and aah to confirm that you were going are we going to going to cancel this effective today",
    "foramen eight to reach you how are you today are you today",
    "everything everything's fine ma'am thank you very much for asking how are you there hours are they going",
    "hey cares how are you today",
    "hmm yes and then if you want to lose the device the device is the weather that are available that will that will show you ma'am",
    "all right all right it's seven seven eight let me know if you're okay just stay on the line aah and aah april all right so april how aah while i process it for you aah how's your day today",
    "by the way by the way i can send the system that the very front of you trying to do have needs to be updated into the fifty five fifty five unlimited core plan which is the same the same thing but the name is going to be updated say it and it needs to be updated for you to basically get a chance to apply to hi to program that we have in this case okay so you mentioned you broke up and if it problem if you would if you would like to applied we can send you the the device form for you to do it but it but that's it basically somethin' else um that you can do you know to have to have the when you get the may plan updated data but what i would need now is is to update the right so what do you think think mister michael because",
    "i'm good thank you so much for asking how are you doing are you doing today",
    "i'm great sir um um how about you",
    "hi baggage good afternoon the afternoon how are you today",
    "during high doreen how are you doing today",
    "morning thomas how are you today or today",
    "hello how are you doing today mary anna anna",
    "hi there clear how are you how are you",
    "i sorry to interrupt aah hi brandon hi brandon how are you doing today",
    "agent how are you today",
    "hello their return richard how are you",
    "how you doing today patricia",
    "hi i'm in the how are you",
    "i'm really good good thanks for asking how about you",
    "hi welcome to tickets nation my name is my name is berna how can i help you today",
    "hmm hi robert how you doing today",
    "also high hi how can i help you",
    "hi there very welcome to the technician is the one let me have your name your name please",
    "hi gary how can i help you today help you today",
    "if you upgrade as per as per checking here you're eligible for a device that hi discount so that means if you plan you plan on upgrading your device you can definitely and i only have a discount as long as you're logged log in your account over cricket wireless dot com com you can choose your device will recover device device if you should be able to receive a discount discount",
    "hi good morning this is this is chris from cricket how can i help you today okay",
    "hi does how does right how can i help you",
    "hi welcome to coordination this is christian christian can i have your name please",
    "hi hi welcome to the technician this is using sure we can have your first name please",
    "hmm just give me a second here i will and i will just check hmm okay uh uh i'm checking here you know that our on your area yeah yeah but all the tower is working fine and active active so let us go ahead and put and then proceed with the troubleshooting of your device so did you try any trouble and the trouble shooting on your device before",
    "okay okay three five six one and just a disclaimer klamer robert if in case in the future okay okay if you would like to reactivate this line you only have only have twenty nine days you're going to pay whatever the whatever the plan price and fifteen dollars for the activation should be the one twenty nine days when able to read would you reactivate the same number but i know you can and get a new sim card and activate a new number instead instead okay",
    "i know you're you're do is actually actually um okay you're you made you made uh the last payment successful was on the on july right",
    "with regard regards to that one we need to check also the area area and see what could have been the reason the reason why it's acting up that way but uh but uh for the payments you've made that that went towards your because you were in the sixty dollar sixty dollar plan before you just changed it it's to the fifty five dollars there's you know",
    "because we have an option for you to the store restart your services we can either process process the whole fifty three dollars and seventeen been cents are we may process a payment arrangement spent which will you know just to just require an initial payment and then you pay the pay the rest on um on on the one the six six",
    "i understand your stand alright so let me give you the instructions on on how to complete the unlock the unlock you know luggage of the device it's very",
    "aah may i know what number are we going to remove it removing this account",
    "aah if you don't mind me asking aah did you may i know the reason why you want to remove the move the numbers on your account",
    "yeah yeah i will pay it as well it as well if i have it yeah my sister is why that why that as well ma'am i can see let me know let me know if you're you were able like to change the password",
    "in the and that way that you can get the the service of a mobile hot spot husband's him between today until and the following payments i know and september the fifteen just for for the line thirty three cents three cents",
    "hmm hmm so you need to update for sure so fair fair so it's asking you to update the system",
    "okay i got to be more than glad to assist you and i'm doing fine thank you for asking this a referral raphaela and how your self",
    "you can try going to a store and or and uh asking for the that information okay i'm okay but you need to take all your um i um i your i d and the account account information and check if they have access to any to an information on their system because as a customer customer service we do not have um the information permission in our system",
    "and i'm doing great thank you for asking how about you",
    "alright hi sir thank you so much thank you so much for patiently waiting um yeah i was able to check here check here aah i really would get an get an error it shows device not um if you don't mind if you don't mind me asking 'cause this phone is uh i'd be way older phone meaning it's it's a non ticket phone ticket phone okay now aah before you for it was activated to cricket where it was activated to read it before",
    "okay uh well yes oh just to confirm novia on the same thing that that one is happening at the moment is that you are that you are trying to access in your cricket account account and uh it's a it's asking for a security question anything anything that is why is that correct",
    "uh yes maybe that ladies yeah what nice to me the thank you for thank you for that information let me go ahead and head and access here and the account account meter click okay okay so now i'm and please ya that now but i'm here in your your account in order for you to access the so because these um i'll be asking for a for a security question let me go ahead me go ahead and send to you a link in the in that link you're gonna be able to update update the information about the the security question and also amount has been been um and then we can go ahead go ahead and try it again for you to access in your senior cricket account so so i'm gonna go ahead and send this link so thanks for you but i cannot dated i mean that i mean that way you're gonna make sure it was a security to request then you have so i'm gonna send to you this link and i send it can you please can you please tell me if you receive it",
    "i'm good thanks for asking so how can i help you today miss is julia",
    "aah continue adding adding uh the additional mobile hot spot on your phone your phone as i'll be asking your patience and aah please give me at least maximum of a minute or two if there too when it gets back everything should be updated dated okay",
    "instead of being while the fifteenth fifteen dollars the system is asking for seven dollars and and fifty cents and starting next next month or your next bill cycle you're gonna pay the full pay the full fifty five",
    "perfect sir so have a good day sir thank you so much for calling trigger",
    "that's good to hear by the way can by the way can i have your phone number that you're calling about about please",
    "aah if you want let me see if we could frequent because supposedly there should be a bridge the see which page extension tension where you can just process at least an initial initial payment for example fifteen dollar initial michelle payments and then your services will be be good and l the twenty six we have that but but i need to further check your account if if you'd be able to um process any payment would you like to do bridge bridge bridgeport or would you like to really process the whole the whole amount",
    "good evening welcome to cricket my name is name is danielle how may i have the pleasure to speak to",
    "good evening",
    "good good",
    "good evening good evening you mean welcome to cricket my name is danielle how may i have i have the pleasure to speak to",
    "good and good evening log on to",
    "all right so i'm glad i was able to able to assist you regarding with this concern kennett kenneth and just found a reminder that the account account is still good to go up until on the twenty third of august and",
    "i'm sorry about that but aah aah for this time aah and the removing of the numbers were going to have this have this removed for good",
    "yes yes okay just give me one second while i'm doing that okay okay i know this is not their whole jack and and rather leaving uh ticket against you so this coming fifteen of september temporary just going to have to pay twenty five dollars okay okay uh since your auto pay or or the credit card in your auto pay was already updated updated you don't have to reno because because on the fourteenth of our august i mean in september it will to process an autopay which is twenty five dollar five dollars only okay",
    "um your your you do the only way to get another phone would be doing an upgrade an upgrade",
    "okay okay okay okay we're going to be doing had trouble troubleshooting your phone phone from my end aah",
    "okay so what we're going to be doing is that",
    "we're gonna go ahead and work with your phone assist may first day i i was just give okay verizon had a different phone number because uh we need we need to to turn off the phone for two minutes two minutes so what we're going to be doing be doing we're gonna turn off your uh we're gonna need connect the call with you offer to be able to turn off your phone for two minutes then you're going",
    "hi how are you doing are you doing today glenda",
    "your you're most welcome and thank you as well family ma'am i i went to i would appreciate you aah you know doing the thing on the website side aah",
    "doing",
    "thank you okay so at this moment this one no we're already reset the password and i'm good and i'm gonna send you this as well so i sent you an send you an add the you should be receiving some text messages messages telling you right let me check one one is going to tell you like to change your pin inger's your security question that's in case that you need to still still change your password and it says that has been already reset reset so let me check",
    "be assigned as then as aah as far as a text message okay",
    "hmm okay i have noticed too is to that yes you made a eighty dollar payment payments on the twenty eighth of july along uh let me just check so far what happen happened on that payment because right right now the account let's do says um you'll see of fifty three dollars and seventeen cents that and so let me check about that months then okay and okay",
    "and how is your day going so far bye",
    "or due date due date okay alrighty and your services everything everything working fine so far",
    "okay okay all right all right make sure to do so far as you go to the start store just type in the website and a request request directly from the um the corporate i'm trying quarters okay",
    "so far so good good how are you",
    "perfect so far so for that one richard um i was happy in a p m m happy right now and i'm able to uh i owe uh i i was able to aah help you with your when you reach paid now and for that one for that one thank you so much for being part of pickup farmin family retried have a great day and stay safe then bye",
    "excellent to adam and my girlfriend go for today was to help with this whole older h as in just concerns did i accomplish that so far",
    "right okay but okay but the the internet's working so far right services or sir working",
    "thank you how's your day going so far going so far today miss shannon",
    "all right so john john high thing for settled here for today anything anything else i can further assist you with",
    "yeah usually we do have um you know standard high third highway time but then it's really good to know that know that i was um able to receive your call and i would oh and i was able to help you see what so again again i assure you that everything is good now you don't oh you don't need to worry about um your device anymore it's completely unlocked and you can use it to um um different device anytime you want okay",
    "a high welcome to cricket like it's frank speaking how can i help you",
    "within the billing cycle so that you have like you have like eleven days to use this fifteen gigabytes or whatever whatever if ever in case you aah aah aah you did that used all of the fifteen gigabytes but aah any remaining data for the for the hot spot during the time it will not move over on the next the next bill okay guess every every every cycle review refresh so high so how about this aah do you have access to you just to your app right now if we could app or or",
    "it's an a station high anastasia patient how is your day so far",
    "a high aah hi aah welcome to cricket frank speed frank speaking how can i help you",
    "remember to think the customer further business a high hi welcome to cricket nation this and this is lovely speaking how can i help how can i help you",
    "already used the three gigabytes right so in this case there's two plan uh three funds that you cannot grito so there's so there's the forty dollar plan the forty dollar print provide provide you at ten gigabytes a bite of high",
    "okay and now i will say oh so i will provide you the option that we so that we have to choose a security question i will need you to select this is like one of the member i made the answer okay so the options are where did you go the first the first time you fully on a plane who is your i have your first bus in what city do you need you majors post or significant other and what was what was the first name of your best friend in high school",
    "it is also not the right for the four digit pin code um if you want we can answer answer here a security question it says here here what was the first name of your best friend and i in high school",
    "okay well let's pull up your pull up your account first together aah are you calling from calling from el a phone number ending seven eight eight eight eight zero",
    "laurie alright thank you aah if you can go back to your my cricket app app",
    "aah we're trying to make sense make some changes",
    "see here i can see your payment that you made aah aah just recently for forty eight dollars there's",
    "aah yes ma'am that covers actually or two or two lines on the account",
    "that way comparing to aah getting the other one ma'am aah aah left behind but anyway if",
    "aah",
    "okay okay so we have fifty five dollar dollar unlimited plan or we also have to have the sixty dollar unlimited the plan aah",
    "okay and that is for which aah uh which rate plan aah which phone number number the four one nine six",
    "okay aah currently currently you both have the thirty dollar hello plan if he switching of fraud aah from thirty to sixty your your monthly will be ninety dollars",
    "all right you're welcome ma'am please thank you so much for calling have a great day stay safe",
    "thank you so much and i'm glad i was and i was able to resolve your concern by uh now so that you can now enjoy the retail services once once again nicole this is berna and thank you purchasing cricket and have a great day bye",
    "cricket wireless is just like sandra have a great day take care",
    "simply by unlocking or can your device and once again johnny thank you so thank you so much for being patient you have a great day but if okay but if you stay safe",
    "okay great great sir thanks for being part of the cricket family remember remember my name is edward was a pleasure to assist you this year and hope you get better of your allergies i just",
    "have a great day",
    "elsa how a wonderful rest of the day the date and great weekend okay",
    "you're welcome i'm glad we're able to resolve it twelve your concern for today kathleen thank you so much how much again for choosing cricket this is arlene and do you have a great day",
    "just a great week quick recap um corban you calling because one because you wanted",
    "great day",
    "um i can pull i can pull up i can pull up the account with the serial number",
    "um april was able to able to pull up an account april oscar check upon checking here this device was activated dated on november seventh seven and has been deactivated on of them november nine and the last the lines of service is uh of this device why is this just three days so that",
    "um um it should be i m e i",
    "um",
    "um hmm",
    "okay and i can help you with that disconnecting this line aah and uh robert okay uh since you you uh paid for this line this line okay either you can uh they uh they can you can use the paid service service okay on this line i can i can set this aah cancellation on on aah the next cycle you also have also have an option you can remove it um it may immediately but there will be no refund aah no credits but you want charge one charge for this line anymore and your bill will be they will gonna lower down as well",
    "eighty dollars hours correct and then uh since we are since we are prepaid to after you made a payment as july next my next payment is supposed to be august and i understand you mentioned you try to me to me to make a payment but those payments you attempted to come to the process yesterday and even on the fifteenth fifteen or a t and those payments were made we're failed payments your bank is declined declining the payment that's why it did not go through so and so this august they'll um um the line become suspended the cost of that but then i checked also a check back the not the eighty dollar payment that you have before you actually actually are in a sixty dollar plan so uh let me just check if check if you have uh any add on here here okay one moment",
    "yeah i uh from what i can see here the payment you you made for eighty dollars last month right right that's because of the hot spot uh you uh you were in a sixty dollar plan before then then you also have a mobile hot spot add on on so um six and sixty dollars plus ten dollars that sounds comes up to seventy dollars right right and then um there there was a reactivation fee when you made a payment the payment on july so that's so that's",
    "supposed to be yes from the from the eighty dollars that you pay for it includes the the reactive agency because your due date three eight three is on the eighteenth before four and then the payment was made on the twenty it does make makes your do every month to the nineteenth um",
    "correct and and that's why your bill also lowers down to down to um fifty three dollars and and seventeen cents once",
    "allowed to get our customers summers card number over the phone that's why we're no longer in you're allowed to separate the numbers because of the payment payment so it depends on the billing cycle michael aah aah for this one for your account account today is the last day of the billing side billing cycle from last month tomorrow",
    "alright let me so happy i was able to work with your account in your account today and help you out with your options again this is hazel thank you for your time calling cricket cricket and have a nice day okay",
    "hello hello april how can i help you today",
    "all right okay anything else anything else for today april before i let you go",
    "welcome to create condition mister here how can i help you today",
    "perfect sir so any notes here to help today",
    "we process this this uh fifty three dollars and seventeen cents moved and moving forward your payment will just be just the fifty five dollars because you were in a we're in a fifty five dollar plan today no longer",
    "hello thank you for calling cregan wireless is so since all extenders speaking how can i assist you today",
    "tell me how can i ask you can i assist you today sir",
    "thank you so much johnny how can i help you can i help you today",
    "on the fine has actually my um start today today my first",
    "all right that's good good so kathleen what we have done today today mysteries accessed fully reset your password for your to the account so you have access to it now and and just a reminder your next due date september september seventeen bill is forty dollars and dollars and you have autopay set up which is very convenient for you it for you is there anything else i can help you you",
    "let's see right but just to make sure can you um can you can you tell me what phone is it today today or is it still",
    "alright alright there we go yep the account is already set here in a brief a bridge peso again at tomorrow right on or before know before the twenty fourth aah forty five dollars and eighty eighty one cents will be the uh ticket amount the letter so you make the payment through the app the six and one or one or the forget wise it talking up so they could pay or you or using your apps it's all set right now okay that'd be was able to set this up here for you today today would there be anything else let me know here",
    "i'm trying to get this get this is valerie who do i have the pleasure of speaking today today",
    "is a pleasure okay okay there's nothing else that i can do for you for today today",
    "all right had or using how may i help you today today",
    "hello joshua how may i help you today today",
    "by the way how may i have the pleasure to be speaking with today today",
    "aah yes you can purchase and phone right phone right now i know website but but the aah upgrade eligibility they will not aah the until till of tober fourteen so if you do it today today you will be charged full month's aah i mean for i mean full payment on the phone price",
    "yes",
    "yes ma'am",
    "the",
    "remember just hang the customer further business yeah my internet's not working on my side my phone in my phones are working size is it it it turned back on or whatever's going on there's one on it",
    "you too",
    "remember to think the customer for their business hmm yeah can you switch me over to billing billing somebody in billing",
    "remember just sent the customer for their business business messinger mary",
    "is",
    "four six four six four six",
    "you",
    "have two",
    "that option",
    "one",
    "hello yeah",
    "press perfect create a new one the one that you can easily remember",
    "that will be that would be under settings",
    "but nobody nobody told me that i have to use cricket in the six months time period",
    "right",
    "the",
    "no",
    "trying to get through on my phone",
    "okay okay i'll be different definitely get in contact with the word the department with this so",
    "hmm",
    "yes ma'am because you because you are on automatic payment",
    "yes ma'am",
    "sure",
    "okay so you said you said you forgot the well he forgot forgot",
    "well",
    "is",
    "remember just sent the customer for their business business oh hmm",
    "and all i did",
    "i'm sorry",
    "the billing cycle go ends oh yeah",
    "everything",
    "me",
    "your text oh go ahead go ahead",
    "repeat that",
    "ma'am no hmm no it's not your mistake it's the phone itself south sometimes",
    "yeah i lose that yeah l a",
    "thank you can you please provide please provide me the card number",
    "yes",
    "but",
    "can i have the four digit pin please",
    "good morning how are you doing",
    "i'm sorry",
    "hmm",
    "hmm",
    "it for",
    "yes",
    "correct fifty dollars enter his this is in charge yes",
    "but",
    "they go through general",
    "what's",
    "hmm",
    "hmm okay so the paper your device is completely you mean ma'am so that you can block them",
    "me",
    "you want me to go on the payment",
    "forty four",
    "yes sir",
    "assistance",
    "yes ma'am",
    "oh a lake charles out valid shop all six oh one four",
    "you very",
    "the number that order already exist",
    "let me see",
    "aah i can i can i i do this every month",
    "how much is the total amount that you pay all your bills bills michael",
    "please be there please bear with me okay verifying the exact unlock code you have would you have so as part of the policy of the unlocked lock code we can",
    "more",
    "yes i am",
    "glad",
    "child",
    "okay and your four digit pin code",
    "oh who when be hoover when unlock it",
    "i i rebooted it wasn't was an x submit",
    "mobile",
    "hmm so since you have should have the wifi in your wifi is working at your home your home so try to connect your phone so that you can that you can update the system of your device",
    "remember just hang the customer for their business um seat or for denny",
    "me",
    "two two this thank you very much for",
    "those",
    "dollars credit in your account account",
    "i",
    "hmm not",
    "oh",
    "the",
    "all right okay all right thank you how about the i c c i d or the i d or the sim card did you change it as well",
    "let me see hold on in ten dollars on the old one",
    "you",
    "oh no",
    "phone are you looking to upgrade to",
    "yeah",
    "for",
    "new sixty sixty dollar plan",
    "well i see the account and i don't and i don't see any card on file to be honest with you mister mister anderson i see that you don't have autopay",
    "you hear",
    "still once",
    "yep i yep i restarted",
    "yeah already in your account so let me go ahead let me go ahead and check the balance that you have right now",
    "i i will walk you through ovaries very so stay on the line let me send you the link first first",
    "nine",
    "no",
    "yeah right",
    "one from that from that number and it still wouldn't let me call you call you guys either",
    "hmm",
    "and also and also does include",
    "okay it's just four digits so what you what would you want me to try",
    "so",
    "after",
    "and i have been with correct cricket oh well fifteen years",
    "hmm hmm",
    "no",
    "your accounts together gather",
    "so",
    "got it thank you you may i please a minute or two just want to want to make sure that you are connected to the struggle trying to start working",
    "yeah",
    "hold",
    "yeah just miss bernadette's i check the",
    "okay let me go ahead and start processing the seeing the payments it's currently loading",
    "um as of now i now i had two but i just had three earlier there",
    "told",
    "okay just stay on the line i am just i'm just working on the account can i put you on a you on a brief hold allowance minutes",
    "hmm",
    "nine",
    "alright there you go",
    "thank you i got this one here and this is for me to for me to attempt to get the account can i have the four digit did it in please",
    "the",
    "hmm hmm",
    "for",
    "hmm",
    "anything",
    "so you're saying so you're saying the motorola that i'm trying to use is not bad not compatible with",
    "well and then i'll have to go there go make those somebody virus they phone so i can make some to make some phone calls calls aah and which is which is ridiculous you know they did they didn't go they were gonna turn stop they could've new sent me a notice no it was saying that they were gonna turn the phone off that's only courtesy they sent me a puppy i thought they sent me a notice last month the month a month and phone and my errors has not been handle want every month they sent me a phone your phone bill is bill is due on your bill is someone telephones so so and then this time you get up and it don't who is all",
    "christine part first",
    "and what was the zip code you put in",
    "my business",
    "to reset my phone oh",
    "correct",
    "tell me that i have a text and it doesn't make any make a note",
    "as in the reservation sure",
    "no",
    "the",
    "i",
    "what's the card number",
    "i said",
    "yeah it's forty one forty one hundred is my husband",
    "to",
    "yeah well yeah kind dot com sir like i have a link the internet on my phone phone but i didn't know if you go if you guys like so like a tablet with but with the phone line on it that would be that that would have unlimited internet or if you guys if you guys so like like a little it'll aah hot spot box type deal do you",
    "it",
    "hundred",
    "under the settings on there about",
    "so",
    "okay can you check if you have received receive any text message",
    "hmm and that's the i m e i for i for the replaced or the",
    "hmm",
    "from this towers so all we just need to do is to is to undergo with settings to check for travel for troubleshooting okay",
    "can you please roger to dial to a different number",
    "okay first lie i do a i do",
    "let me give you some okay so give me give me what it will be do you have do you have any",
    "oh so",
    "yeah the payments so and so any posted in different request stephanie very bad service",
    "oh",
    "it don't matter no",
    "all righty miss came up let's try the phone the phone on okay",
    "any color i won't correct",
    "so",
    "hmm",
    "the process",
    "is there",
    "okay perfect perfect yep so that whatever everything code you enter there is the note ten code and go d on the account so",
    "hmm",
    "yeah",
    "got",
    "hmm",
    "hmm",
    "mm hmm",
    "so when so when you use up that percentage of the the increment of the fifteen",
    "i i just need your four digit pin please then please",
    "they got the they got the wrong card number",
    "seven",
    "make a phone call",
    "what i'm saying that they did give you a mother give me aah oh like that is why they on it unlocked we don't like the phone aah",
    "is",
    "wait",
    "hmm yes i do",
    "hmm",
    "yeah it did one",
    "is to put your phone your phone number and the amount the card information metion and there will be no charges for that for that",
    "hmm sorry sorry",
    "ma'am yeah",
    "yes ma'am",
    "m e i what you said my name",
    "you",
    "i'm sorry",
    "the phone number",
    "so so basic basically and just to confirm may sir may i please have the i m e i of the may i have the phone",
    "connection"
])


actual = [
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Interaction Greeting",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Customer Positivity",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "Agent Rapport",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS",
    "OTHERS"
]

f1 = f1_score(actual, preds, average='macro')

acc = accuracy_score(actual, preds)

print([runtime, f1, acc])

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/50 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 2000
  Num epochs = 5
  Total optimization steps = 835
  Total train batch size = 12
running astray


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/167 [00:00<?, ?it/s]

Iteration:   0%|          | 0/167 [00:00<?, ?it/s]

Iteration:   0%|          | 0/167 [00:00<?, ?it/s]

Iteration:   0%|          | 0/167 [00:00<?, ?it/s]

Iteration:   0%|          | 0/167 [00:00<?, ?it/s]

[79.438880001, 0.6317989868039713, 0.6296851574212894]


In [5]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import spacy
import string
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_trainset_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_testset_mod.xlsx")

nlp = spacy.load("en_core_web_sm")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    sentence = " ".join(mytokens)

    return sentence

train_ds['tokenize'] = train_ds['Text'].apply(spacy_tokenizer)
test_ds['tokenize'] = test_ds['Text'].apply(spacy_tokenizer)

train_ds['embeddings'] = train_ds['tokenize'].apply(model.encode)

X_train = train_ds['embeddings'].to_list()
y_train = train_ds['Class'].to_list()

from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier()
MLP.fit(X_train,y_train)

test_ds['embeddings'] = test_ds['tokenize'].apply(model.encode)
X_test = test_ds['embeddings'].to_list()
y_test = test_ds['Class'].to_list()

from sklearn import metrics
predicted = MLP.predict(X_test)
print("MLP Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("MLP Regression Precision:",metrics.precision_score(y_test, predicted, average='macro'))
print("MLP Regression Recall:",metrics.recall_score(y_test, predicted, average='macro'))

/home/ubuntu/nilay/opt-venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Regression Accuracy: 0.7142857142857143
MLP Regression Precision: 0.5992063492063492
MLP Regression Recall: 0.6595238095238096


In [2]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import spacy
import string
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_test_shot.xlsx")

nlp = spacy.load("en_core_web_sm")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    sentence = " ".join(mytokens)

    return sentence

train_ds['tokenize'] = train_ds['Text'].apply(spacy_tokenizer)
test_ds['tokenize'] = test_ds['Text'].apply(spacy_tokenizer)

train_ds['embeddings'] = train_ds['tokenize'].apply(model.encode)

X_train = train_ds['embeddings'].to_list()
y_train = train_ds['Class'].to_list()

from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier()
MLP.fit(X_train,y_train)

test_ds['embeddings'] = test_ds['tokenize'].apply(model.encode)
X_test = test_ds['embeddings'].to_list()
y_test = test_ds['Class'].to_list()

from sklearn import metrics
predicted = MLP.predict(X_test)
print("MLP Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("MLP Regression Precision:",metrics.precision_score(y_test, predicted, average='macro'))
print("MLP Regression Recall:",metrics.recall_score(y_test, predicted, average='macro'))

MLP Regression Accuracy: 0.5952023988005997
MLP Regression Precision: 0.617327106560457
MLP Regression Recall: 0.6044374413034553


In [12]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import spacy
import string
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_test_shot.xlsx")

nlp = spacy.load("en_core_web_sm")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    sentence = " ".join(mytokens)

    return sentence

train_ds['tokenize'] = train_ds['Text'].apply(spacy_tokenizer)
test_ds['tokenize'] = test_ds['Text'].apply(spacy_tokenizer)

train_ds['embeddings'] = train_ds['tokenize'].apply(model.encode)

X_train = train_ds['embeddings'].to_list()
y_train = train_ds['Class'].to_list()

from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier()
MLP.fit(X_train,y_train)

test_ds['embeddings'] = test_ds['tokenize'].apply(model.encode)
X_test = test_ds['embeddings'].to_list()
y_test = test_ds['Class'].to_list()

from sklearn import metrics
predicted = MLP.predict(X_test)
print("MLP Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("MLP Regression Precision:",metrics.precision_score(y_test, predicted, average='macro'))
print("MLP Regression Recall:",metrics.recall_score(y_test, predicted, average='macro'))

/home/ubuntu/nilay/opt-venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Regression Accuracy: 0.6101949025487257
MLP Regression Precision: 0.6324764143909669
MLP Regression Recall: 0.6149604277749455


In [14]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import spacy
import string
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_trainset_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_testset_mod.xlsx")

nlp = spacy.load("en_core_web_sm")

train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    sentence = " ".join(mytokens)

    return sentence

train_ds['tokenize'] = train_ds['Text'].apply(spacy_tokenizer)
test_ds['tokenize'] = test_ds['Text'].apply(spacy_tokenizer)

train_ds['embeddings'] = train_ds['tokenize'].apply(model.encode)

X_train = train_ds['embeddings'].to_list()
y_train = train_ds['Class'].to_list()

from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier()
MLP.fit(X_train,y_train)

test_ds['embeddings'] = test_ds['tokenize'].apply(model.encode)
X_test = test_ds['embeddings'].to_list()
y_test = test_ds['Class'].to_list()

from sklearn import metrics
predicted = MLP.predict(X_test)
print("MLP Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("MLP Regression Precision:",metrics.precision_score(y_test, predicted, average='macro'))
print("MLP Regression Recall:",metrics.recall_score(y_test, predicted, average='macro'))

/home/ubuntu/nilay/opt-venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Regression Accuracy: 0.7857142857142857
MLP Regression Precision: 0.7256235827664399
MLP Regression Recall: 0.7095238095238096


In [11]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import spacy
import string
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_test_shot.xlsx")

nlp = spacy.load("en_core_web_sm")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    sentence = " ".join(mytokens)

    return sentence

train_ds['tokenize'] = train_ds['Text'].apply(spacy_tokenizer)
test_ds['tokenize'] = test_ds['Text'].apply(spacy_tokenizer)

train_ds['embeddings'] = train_ds['tokenize'].apply(model.encode)

X_train = train_ds['embeddings'].to_list()
y_train = train_ds['Class'].to_list()

from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier()
MLP.fit(X_train,y_train)

test_ds['embeddings'] = test_ds['tokenize'].apply(model.encode)
X_test = test_ds['embeddings'].to_list()
y_test = test_ds['Class'].to_list()

from sklearn import metrics
predicted = MLP.predict(X_test)
print("MLP Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("MLP Regression Precision:",metrics.precision_score(y_test, predicted, average='macro'))
print("MLP Regression Recall:",metrics.recall_score(y_test, predicted, average='macro'))

MLP Regression Accuracy: 0.5982008995502249
MLP Regression Precision: 0.6197006142712249
MLP Regression Recall: 0.6061246249529726


In [13]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import spacy
import string
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_trainset_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_testset_mod.xlsx")

nlp = spacy.load("en_core_web_sm")

train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    sentence = " ".join(mytokens)

    return sentence

train_ds['tokenize'] = train_ds['Text'].apply(spacy_tokenizer)
test_ds['tokenize'] = test_ds['Text'].apply(spacy_tokenizer)

train_ds['embeddings'] = train_ds['tokenize'].apply(model.encode)

X_train = train_ds['embeddings'].to_list()
y_train = train_ds['Class'].to_list()

from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier()
MLP.fit(X_train,y_train)

test_ds['embeddings'] = test_ds['tokenize'].apply(model.encode)
X_test = test_ds['embeddings'].to_list()
y_test = test_ds['Class'].to_list()

from sklearn import metrics
predicted = MLP.predict(X_test)
print("MLP Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("MLP Regression Precision:",metrics.precision_score(y_test, predicted, average='macro'))
print("MLP Regression Recall:",metrics.recall_score(y_test, predicted, average='macro'))

/home/ubuntu/nilay/opt-venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Regression Accuracy: 0.7142857142857143
MLP Regression Precision: 0.5992063492063492
MLP Regression Recall: 0.6595238095238096


In [2]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import spacy
import string
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

train_ds = pd.read_csv("/home/ubuntu/nilay/Datasets/cricket_final_train_df.csv")
test_ds = pd.read_csv("/home/ubuntu/nilay/Datasets/processed_df_08-16-2021_13-21-00_sid_1675508193_dbsid_504.csv")

nlp = spacy.load("en_core_web_sm")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    sentence = " ".join(mytokens)

    return sentence

train_ds['tokenize'] = train_ds['Text'].apply(spacy_tokenizer)
test_ds['tokenize'] = test_ds['Text'].apply(spacy_tokenizer)

train_ds['embeddings'] = train_ds['tokenize'].apply(model.encode)

X_train = train_ds['embeddings'].to_list()
y_train = train_ds['Class'].to_list()

from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier()
MLP.fit(X_train,y_train)

test_ds['embeddings'] = test_ds['tokenize'].apply(model.encode)
X_test = test_ds['embeddings'].to_list()
y_test = test_ds['Class'].to_list()

from sklearn import metrics
predicted = MLP.predict(X_test)
print("MLP Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("MLP Regression Precision:",metrics.precision_score(y_test, predicted, average='macro'))
print("MLP Regression Recall:",metrics.recall_score(y_test, predicted, average='macro'))

/home/ubuntu/nilay/opt-venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Regression Accuracy: 0.45
MLP Regression Precision: 0.39333333333333337
MLP Regression Recall: 0.5787878787878789


/home/ubuntu/nilay/opt-venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import spacy
import string
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-large-en-v1.5')

train_ds = pd.read_csv("/home/ubuntu/nilay/Datasets/cricket_final_train_df.csv")
test_ds = pd.read_csv("/home/ubuntu/nilay/Datasets/processed_df_08-16-2021_13-21-00_sid_1675508193_dbsid_504.csv")

nlp = spacy.load("en_core_web_sm")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    sentence = " ".join(mytokens)

    return sentence

train_ds['tokenize'] = train_ds['Text'].apply(spacy_tokenizer)
test_ds['tokenize'] = test_ds['Text'].apply(spacy_tokenizer)

train_ds['embeddings'] = train_ds['tokenize'].apply(model.encode)

X_train = train_ds['embeddings'].to_list()
y_train = train_ds['Class'].to_list()

from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier()
MLP.fit(X_train,y_train)

test_ds['embeddings'] = test_ds['tokenize'].apply(model.encode)
X_test = test_ds['embeddings'].to_list()
y_test = test_ds['Class'].to_list()

from sklearn import metrics
predicted = MLP.predict(X_test)
print("MLP Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("MLP Regression Precision:",metrics.precision_score(y_test, predicted, average='macro'))
print("MLP Regression Recall:",metrics.recall_score(y_test, predicted, average='macro'))

/home/ubuntu/nilay/opt-venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Regression Accuracy: 0.775
MLP Regression Precision: 0.5333333333333334
MLP Regression Recall: 0.6575757575757576


/home/ubuntu/nilay/opt-venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import spacy
import string
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-large-en-v1.5')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_trainset_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/cricket_testset_mod.xlsx")

nlp = spacy.load("en_core_web_sm")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    sentence = " ".join(mytokens)

    return sentence

train_ds['tokenize'] = train_ds['Text'].apply(spacy_tokenizer)
test_ds['tokenize'] = test_ds['Text'].apply(spacy_tokenizer)

train_ds['embeddings'] = train_ds['tokenize'].apply(model.encode)

X_train = train_ds['embeddings'].to_list()
y_train = train_ds['Class'].to_list()

from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier()
MLP.fit(X_train,y_train)

test_ds['embeddings'] = test_ds['tokenize'].apply(model.encode)
X_test = test_ds['embeddings'].to_list()
y_test = test_ds['Class'].to_list()

from sklearn import metrics
predicted = MLP.predict(X_test)
print("MLP Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("MLP Regression Precision:",metrics.precision_score(y_test, predicted, average='macro'))
print("MLP Regression Recall:",metrics.recall_score(y_test, predicted, average='macro'))

`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.
/home/ubuntu/nilay/opt-venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Regression Accuracy: 0.7142857142857143
MLP Regression Precision: 0.6122448979591837
MLP Regression Recall: 0.7142857142857143


In [2]:
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from peft import LoraConfig
from peft import TaskType
from peft import get_peft_model
from datasets import Dataset
import time
from sklearn.metrics import f1_score, accuracy_score
import spacy
import string
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-large-en-v1.5')

train_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_fewshot.xlsx")
test_ds = pd.read_excel("/home/ubuntu/nilay/Datasets/labelled_data_test_shot.xlsx")

nlp = spacy.load("en_core_web_sm")


train_dataset = Dataset.from_pandas(train_ds)
test_dataset = Dataset.from_pandas(test_ds)

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    sentence = " ".join(mytokens)

    return sentence

train_ds['tokenize'] = train_ds['Text'].apply(spacy_tokenizer)
test_ds['tokenize'] = test_ds['Text'].apply(spacy_tokenizer)

train_ds['embeddings'] = train_ds['tokenize'].apply(model.encode)

X_train = train_ds['embeddings'].to_list()
y_train = train_ds['Class'].to_list()

from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier()
MLP.fit(X_train,y_train)

test_ds['embeddings'] = test_ds['tokenize'].apply(model.encode)
X_test = test_ds['embeddings'].to_list()
y_test = test_ds['Class'].to_list()

from sklearn import metrics
predicted = MLP.predict(X_test)
print("MLP Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("MLP Regression Precision:",metrics.precision_score(y_test, predicted, average='macro'))
print("MLP Regression Recall:",metrics.recall_score(y_test, predicted, average='macro'))

MLP Regression Accuracy: 0.6281859070464768
MLP Regression Precision: 0.657016725272219
MLP Regression Recall: 0.630177898629377
